### Raw Hate Speech (HS) Data - Parsing, Preprocessing and Text Analysis

# Data Frequency Analysis 
## - as nxgram analysis
## - as 2-gram, 3-gram, and 4-gram analysis

The scripts were created and published by Merja Kreivi-Kauppinen, and
are part of research work carried in University of Oulu in 2020-2023.

The study is reported on (Master’s Thesis) research paper - 
Merja Kreivi-Kauppinen (2024) Hate Speech Detection of Dialectal, Granular and Urban Finnish. 
University of Oulu, Degree Programme in Computer Science and Engineering. Master’s Thesis.

### Import python packages and libraries

In [1]:
# import python packages and libraries
import sys, re, os, openpyxl
import json
import time, datetime, random, string
import numpy as np
import pandas as pd

import sklearn
print('The scikit-learn version is {}.'.format(sklearn.__version__))
from sklearn.feature_extraction.text import CountVectorizer

# import NLTK methods and libraries
import nltk
#nltk.download('punkt')
#nltk.download('stopwords')
#from nltk.stem.snowball import SnowballStemmer
from nltk.tokenize import casual_tokenize, word_tokenize
from nltk.corpus import stopwords
# import NLTK ngrams to group words in twograms, trigrams, fourgrams etc.
from nltk import ngrams
# import counter to count the words / tokens / lemmas
from collections import Counter

import matplotlib.pyplot as plt
from matplotlib import rc
import seaborn as sns
from pylab import rcParams

# set seaborn figures
%matplotlib inline
rcParams['figure.figsize'] = 8, 4
sns.set(style='darkgrid', palette='muted', font_scale=1.0)

The scikit-learn version is 1.1.2.


## Download HS Dataset for Analysis

### Download manually annotated collection HS data

In [2]:
# import data from xlsx file to pd dataFolder
cwd = os.getcwd()
folder = '\\data\\'
#csv_file = 'HSTrainData.xlsx'
csv_file = 'Manually_Annotated_Collection_ALL_FINAL_22023Merja.xlsx'
csv_source = cwd + folder + csv_file
#print(csv_source)
df = pd.read_excel(csv_source)
df

,id,sample,sentiment,polarity,HSbinary,HSstrength,HStarget,HStopic,HSform,emotion,urban_finnish,correct_finnish,user_nick
0,1,- Ajatus siitä että kaikki henkilön tienaamat...,negative,-1,False,0,NaN,NaN,NaN,"UNPLEASENT, ANTICIPATION CRITICAL, SARCASTIC",- Ajatus siitä että kaikki henkilön tienaamat...,- Ajatus siitä että kaikki henkilön tienaamat...,Meria
1,2,"- Kaivovertauksessa, viime hallitus on kaivan...",negative,-2,False,0,NaN,NaN,NaN,"NEUTRAL NONE, UNPLEASENT, ANTICIPATION CRITICAL","- Kaivovertauksessa, viime hallitus on kaivan...","- Kaivovertauksessa, viime hallitus on kaivan...",Meria
2,3,--´973´¤-.ttu,negative,-5,True,2,NONE,OTHER,"SWEARING, GRANULATED",CONTEMPT DISRESPECT,--´973´¤-.ttu,973 vittu,Meria
3,4,-=>Widdu joo<=-,negative,-4,True,2,NONE,TROLLING,"JOKE SARCASM, SWEARING, GRANULATED",SARCASTIC,-=> Widdu joo <=-,-=> vittu joo <=-,Meria
4,5,"- Ei , mutta olen joutunut elämään katsomalla ...",negative,-5,False,0,NaN,NaN,NaN,"SADNESS, FEAR","- Ei , mutta olen joutunut elämään katsomalla ...","Ei , mutta olen joutunut elämään katsomalla ku...",Meria
...,...,...,...,...,...,...,...,...,...,...,...,...,...
6624,6625,😂😂😂😂😂😂😂😂😂😂😂😂😂😂,positive,5,False,0,NaN,NaN,NaN,JOY,😂😂😂😂😂😂😂😂😂😂😂😂😂😂,😂😂😂😂😂,Meria
6625,6626,😠👉👩‍💼,negative,-3,False,0,NaN,NaN,NaN,ANGER HATE,😠👉👩‍💼,😠👉👩‍💼,Meria
6626,6627,🤣🤣🤣🤣🤣🤣🤣🤣🤣🤣,positive,5,False,0,NaN,NaN,NaN,JOY,🤣🤣🤣🤣🤣🤣🤣🤣🤣🤣,🤣🤣🤣🤣🤣🤣🤣🤣🤣🤣,Meria
6627,6628,🤮😡,negative,-4,False,0,NaN,NaN,NaN,"DISGUST, ANGER HATE",🤮😡,🤮😡,Meria


In [3]:
# Change text samples to string
df["stringSample"] = df[str("sample")]
df["stringCorrect"] = df[str("correct_finnish")]

# add string len column
df["stringLength"] = ""

# extract string length measures of data
def extract_stringLen(df):
    for item in df["stringCorrect"]:
        #newframe["stringLength"] = len(str(item))
        df["stringLength"] = df.stringCorrect.str.len()
    return df

extract_stringLen(df)
df.head(50)

,id,sample,sentiment,polarity,HSbinary,HSstrength,HStarget,HStopic,HSform,emotion,urban_finnish,correct_finnish,user_nick,stringSample,stringCorrect,stringLength
0,1,- Ajatus siitä että kaikki henkilön tienaamat...,negative,-1,False,0,NaN,NaN,NaN,"UNPLEASENT, ANTICIPATION CRITICAL, SARCASTIC",- Ajatus siitä että kaikki henkilön tienaamat...,- Ajatus siitä että kaikki henkilön tienaamat...,Meria,- Ajatus siitä että kaikki henkilön tienaamat...,- Ajatus siitä että kaikki henkilön tienaamat...,866
1,2,"- Kaivovertauksessa, viime hallitus on kaivan...",negative,-2,False,0,NaN,NaN,NaN,"NEUTRAL NONE, UNPLEASENT, ANTICIPATION CRITICAL","- Kaivovertauksessa, viime hallitus on kaivan...","- Kaivovertauksessa, viime hallitus on kaivan...",Meria,"- Kaivovertauksessa, viime hallitus on kaivan...","- Kaivovertauksessa, viime hallitus on kaivan...",984
2,3,--´973´¤-.ttu,negative,-5,True,2,NONE,OTHER,"SWEARING, GRANULATED",CONTEMPT DISRESPECT,--´973´¤-.ttu,973 vittu,Meria,--´973´¤-.ttu,973 vittu,9
3,4,-=>Widdu joo<=-,negative,-4,True,2,NONE,TROLLING,"JOKE SARCASM, SWEARING, GRANULATED",SARCASTIC,-=> Widdu joo <=-,-=> vittu joo <=-,Meria,-=>Widdu joo<=-,-=> vittu joo <=-,17
4,5,"- Ei , mutta olen joutunut elämään katsomalla ...",negative,-5,False,0,NaN,NaN,NaN,"SADNESS, FEAR","- Ei , mutta olen joutunut elämään katsomalla ...","Ei , mutta olen joutunut elämään katsomalla ku...",Meria,"- Ei , mutta olen joutunut elämään katsomalla ...","Ei , mutta olen joutunut elämään katsomalla ku...",144
5,6,- ei viddu -,negative,-5,True,2,NONE,"OPINION, TROLLING","SWEARING, GRANULATED",SURPRISE,ei viddu,ei vittu,Meria,- ei viddu -,ei vittu,8
6,7,- Huvittunut -,positive,1,False,0,NaN,NaN,NaN,JOY,Huvittunut,huvittunut,Meria,- Huvittunut -,huvittunut,10
7,8,"- Hyvä että JOKU näkee uuden , tyhmän veroperu...",negative,-4,True,1,NONE,OPINION,SWEARING,"MIXED EMOTIONS, JOY, ANTICIPATION CRITICAL","- Hyvä että JOKU näkee uuden , tyhmän veroperu...","Hyvä että JOKU näkee uuden, tyhmän veroperuste...",Meria,"- Hyvä että JOKU näkee uuden , tyhmän veroperu...","Hyvä että JOKU näkee uuden, tyhmän veroperuste...",124
8,9,- Kukaan luterilainen ei korota itseään hänen ...,negative,-2,False,0,NaN,NaN,NaN,SADNESS,- Kukaan luterilainen ei korota itseään hänen ...,Kukaan luterilainen ei korota itseään hänen su...,Meria,- Kukaan luterilainen ei korota itseään hänen ...,Kukaan luterilainen ei korota itseään hänen su...,105
9,10,- kuten on sovittu -,neutral,0,False,0,NaN,NaN,NaN,NEUTRAL NONE,- kuten on sovittu -,kuten on sovittu,Meria,- kuten on sovittu -,kuten on sovittu,16


In [4]:
# change sentiment values
df['sentiment'] = df['sentiment'].replace('neutral', 0)
df['sentiment'] = df['sentiment'].replace('positive', 1)
df['sentiment'] = df['sentiment'].replace('negative', 2)
df.head(50)

,id,sample,sentiment,polarity,HSbinary,HSstrength,HStarget,HStopic,HSform,emotion,urban_finnish,correct_finnish,user_nick,stringSample,stringCorrect,stringLength
0,1,- Ajatus siitä että kaikki henkilön tienaamat...,2,-1,False,0,NaN,NaN,NaN,"UNPLEASENT, ANTICIPATION CRITICAL, SARCASTIC",- Ajatus siitä että kaikki henkilön tienaamat...,- Ajatus siitä että kaikki henkilön tienaamat...,Meria,- Ajatus siitä että kaikki henkilön tienaamat...,- Ajatus siitä että kaikki henkilön tienaamat...,866
1,2,"- Kaivovertauksessa, viime hallitus on kaivan...",2,-2,False,0,NaN,NaN,NaN,"NEUTRAL NONE, UNPLEASENT, ANTICIPATION CRITICAL","- Kaivovertauksessa, viime hallitus on kaivan...","- Kaivovertauksessa, viime hallitus on kaivan...",Meria,"- Kaivovertauksessa, viime hallitus on kaivan...","- Kaivovertauksessa, viime hallitus on kaivan...",984
2,3,--´973´¤-.ttu,2,-5,True,2,NONE,OTHER,"SWEARING, GRANULATED",CONTEMPT DISRESPECT,--´973´¤-.ttu,973 vittu,Meria,--´973´¤-.ttu,973 vittu,9
3,4,-=>Widdu joo<=-,2,-4,True,2,NONE,TROLLING,"JOKE SARCASM, SWEARING, GRANULATED",SARCASTIC,-=> Widdu joo <=-,-=> vittu joo <=-,Meria,-=>Widdu joo<=-,-=> vittu joo <=-,17
4,5,"- Ei , mutta olen joutunut elämään katsomalla ...",2,-5,False,0,NaN,NaN,NaN,"SADNESS, FEAR","- Ei , mutta olen joutunut elämään katsomalla ...","Ei , mutta olen joutunut elämään katsomalla ku...",Meria,"- Ei , mutta olen joutunut elämään katsomalla ...","Ei , mutta olen joutunut elämään katsomalla ku...",144
5,6,- ei viddu -,2,-5,True,2,NONE,"OPINION, TROLLING","SWEARING, GRANULATED",SURPRISE,ei viddu,ei vittu,Meria,- ei viddu -,ei vittu,8
6,7,- Huvittunut -,1,1,False,0,NaN,NaN,NaN,JOY,Huvittunut,huvittunut,Meria,- Huvittunut -,huvittunut,10
7,8,"- Hyvä että JOKU näkee uuden , tyhmän veroperu...",2,-4,True,1,NONE,OPINION,SWEARING,"MIXED EMOTIONS, JOY, ANTICIPATION CRITICAL","- Hyvä että JOKU näkee uuden , tyhmän veroperu...","Hyvä että JOKU näkee uuden, tyhmän veroperuste...",Meria,"- Hyvä että JOKU näkee uuden , tyhmän veroperu...","Hyvä että JOKU näkee uuden, tyhmän veroperuste...",124
8,9,- Kukaan luterilainen ei korota itseään hänen ...,2,-2,False,0,NaN,NaN,NaN,SADNESS,- Kukaan luterilainen ei korota itseään hänen ...,Kukaan luterilainen ei korota itseään hänen su...,Meria,- Kukaan luterilainen ei korota itseään hänen ...,Kukaan luterilainen ei korota itseään hänen su...,105
9,10,- kuten on sovittu -,0,0,False,0,NaN,NaN,NaN,NEUTRAL NONE,- kuten on sovittu -,kuten on sovittu,Meria,- kuten on sovittu -,kuten on sovittu,16


## Download Spacy + Voikko Method

### spacy-fi asennus ohjeet

Asenna spacy 

    - pip install -U pip setuptools wheel
    
    - pip install -U spacy

Install the English language model by running:

    - python -m spacy download en_core_web_sm

Install the Finnish language model by running:

    - pip install spacy_fi_experimental_web_md

In [5]:
# Import and check Spacy
import os
import spacy
print(f"Spacy Version: {spacy.__version__}")

Spacy Version: 3.5.0


In [6]:
# Test large Finnish model

# Load large Finnish model
FinSpacyLarge = spacy.load("fi_core_news_lg")

# Test the model
doc = FinSpacyLarge('Terveyden ja hyvinvoinnin laitos (THL) johtaa hanketta. Ilmassa on kevättä. Rännit tippuvat ja katuojat juoksevat.')
for t in doc:
    print(f'{t.lemma_}\t{t.pos_}')

terveys	NOUN
ja	CCONJ
hyvinvointi	NOUN
laitos	NOUN
(	PUNCT
THL	NOUN
)	PUNCT
johtaa	VERB
hanke	NOUN
.	PUNCT
Ilmassa	NOUN
olla	AUX
kevät	NOUN
.	PUNCT
Rännit	NOUN
tippua	VERB
ja	CCONJ
katuoja	NOUN
juoksevat	VERB
.	PUNCT


### Voikko asennus ohjeet

Voikko asennusohjeet pythonille windowsilla

    https://voikko.puimula.org/python.html

Asenna libvoikko anacondalle haluamaasi kansioon komennolla

    - pip3 install libvoikko

Rakenna Voikko kansio esimerkiksi suoraan C: kansioon

C:/Voikko/

    - libvoikko-1.dll
    
    - libvoikko.dll.a
    
    - libvoikko-4.3+win1.tar.gz
    
    - 5/(v5 VFST sanasto kansiot auki zipattuna)

Voikko sanastot kansiot löytyvät osoitteesta

    https://www.puimula.org/htp/testing/voikko-snapshot-v5/

In [7]:
from libvoikko import Voikko

In [8]:
# set up Voikko search path for Voikko help
Voikko.setLibrarySearchPath("c:/Voikko")
help(Voikko)

Help on class Voikko in module libvoikko:

class Voikko(builtins.object)
 |  Voikko(language, path=None)
 |  
 |  Represents an instance of Voikko. The instance has state, such as
 |  settings related to spell checking and hyphenation, and methods for performing
 |  various natural language analysis operations. One instance should not be
 |  used simultaneously from multiple threads.
 |  
 |  Methods defined here:
 |  
 |  __del__(self)
 |  
 |  __init__(self, language, path=None)
 |      Creates a new Voikko instance with the following optional parameters:
 |      language  BCP 47 language tag to be used.
 |      path      Extra path that will be checked first when looking for linguistic
 |                resources.
 |  
 |  analyze(self, word)
 |      Analyze the morphology of given word and return the list of
 |      analysis results. The results are represented as maps having property
 |      names as keys and property values as values.
 |  
 |  attributeValues(self, attributeName)

In [9]:
# get Voikko directory list
print("\nVoikko directory list:  ", Voikko.listDicts("C:/Voikko"))


Voikko directory list:   [<fi,,standard,suomi (perussanasto)>, <fi,,dialects,suomi (murteellisten, vanhojen ja harvinaisten sanojen sanasto)>, <fi,,education,suomi (kasvatustieteen sanasto)>, <fi,,medicine,suomi (matematiikan, fysiikan, kemian, biologian, maantieteen, geologian ja lääketieteen sanastot)>, <fi,,morpho,suomi, erittäin laaja sanasto (mukana myös morfologisessa analyysissä tarvittava lisäinformaatio)>, <fi,,morphoid,suomi, erittäin laaja sanasto (mukana myös morfologisessa analyysissä tarvittava lisäinformaatio ja linkitys Joukahaiseen)>, <fi,,po,suomi (po-tiedostojen oikolukusanasto)>, <fi,,science,suomi (matematiikan, fysiikan, kemian, biologian, maantieteen, geologian, lääketieteen, kasvatustieteen ja tietotekniikan sanastot)>]


In [10]:
# get Voikko version
print("Voikko version:  ", Voikko.getVersion())

Voikko version:   4.3


In [11]:
print("\nCurrent Working Directory " , os.getcwd())


Current Working Directory  C:\Users\merja\Desktop\HS_research_project\DataAnalysis


In [12]:
# Test experimental Finnish Spacy model with Spacy

# Load large Finnish model
FinSpacyExp = spacy.load("spacy_fi_experimental_web_md")

# Test the model
doc = FinSpacyExp('Terveyden ja hyvinvoinnin laitos (THL) johtaa hanketta. Ilmassa on kevättä. Rännit tippuvat ja katuojat juoksevat.')
for t in doc:
    print(f'{t.lemma_}\t{t.pos_}')

terveys	NOUN
ja	CCONJ
hyvinvointi	NOUN
laitos	NOUN
(	PUNCT
THL	PROPN
)	PUNCT
johtaa	VERB
hanke	NOUN
.	PUNCT
ilma	NOUN
olla	AUX
kevät	NOUN
.	PUNCT
ränni	NOUN
tippua	VERB
ja	CCONJ
katuoja	NOUN
juosta	VERB
.	PUNCT


## Testing lemmatization with Experimental Finnish Spacy+Voikko method

In [13]:
# Lemmatization of raw samples
df['sampleLemmas'] = df['stringSample'].apply(lambda row: " ".join([token.lemma_ for token in FinSpacyExp(row)]))

# Lemmatization of corrected samples
df['correctLemmas'] = df['stringCorrect'].apply(lambda row: " ".join([token.lemma_ for token in FinSpacyExp(row)]))
df.head(10)

,id,sample,sentiment,polarity,HSbinary,HSstrength,HStarget,HStopic,HSform,emotion,urban_finnish,correct_finnish,user_nick,stringSample,stringCorrect,stringLength,sampleLemmas,correctLemmas
0,1,- Ajatus siitä että kaikki henkilön tienaamat...,2,-1,False,0,NaN,NaN,NaN,"UNPLEASENT, ANTICIPATION CRITICAL, SARCASTIC",- Ajatus siitä että kaikki henkilön tienaamat...,- Ajatus siitä että kaikki henkilön tienaamat...,Meria,- Ajatus siitä että kaikki henkilön tienaamat...,- Ajatus siitä että kaikki henkilön tienaamat...,866,- ajatus se että kaikki henkilö tienata tulo...,- ajatus se että kaikki henkilö tienata tulo...
1,2,"- Kaivovertauksessa, viime hallitus on kaivan...",2,-2,False,0,NaN,NaN,NaN,"NEUTRAL NONE, UNPLEASENT, ANTICIPATION CRITICAL","- Kaivovertauksessa, viime hallitus on kaivan...","- Kaivovertauksessa, viime hallitus on kaivan...",Meria,"- Kaivovertauksessa, viime hallitus on kaivan...","- Kaivovertauksessa, viime hallitus on kaivan...",984,"- kaivovertaus , viime hallitus olla kaivaa ...","- kaivovertaus , viime hallitus olla kaivaa ..."
2,3,--´973´¤-.ttu,2,-5,True,2,NONE,OTHER,"SWEARING, GRANULATED",CONTEMPT DISRESPECT,--´973´¤-.ttu,973 vittu,Meria,--´973´¤-.ttu,973 vittu,9,--´973´¤-.ttu,973 vittu
3,4,-=>Widdu joo<=-,2,-4,True,2,NONE,TROLLING,"JOKE SARCASM, SWEARING, GRANULATED",SARCASTIC,-=> Widdu joo <=-,-=> vittu joo <=-,Meria,-=>Widdu joo<=-,-=> vittu joo <=-,17,-=>Widdu joo<= -,-= > vittu joo < = -
4,5,"- Ei , mutta olen joutunut elämään katsomalla ...",2,-5,False,0,NaN,NaN,NaN,"SADNESS, FEAR","- Ei , mutta olen joutunut elämään katsomalla ...","Ei , mutta olen joutunut elämään katsomalla ku...",Meria,"- Ei , mutta olen joutunut elämään katsomalla ...","Ei , mutta olen joutunut elämään katsomalla ku...",144,"- ei , mutta olla joutua elää katsoa kuinka ki...","ei , mutta olla joutua elää katsoa kuinka kius..."
5,6,- ei viddu -,2,-5,True,2,NONE,"OPINION, TROLLING","SWEARING, GRANULATED",SURPRISE,ei viddu,ei vittu,Meria,- ei viddu -,ei vittu,8,- ei viddu -,ei vittu
6,7,- Huvittunut -,1,1,False,0,NaN,NaN,NaN,JOY,Huvittunut,huvittunut,Meria,- Huvittunut -,huvittunut,10,- huvittunut -,huvittua
7,8,"- Hyvä että JOKU näkee uuden , tyhmän veroperu...",2,-4,True,1,NONE,OPINION,SWEARING,"MIXED EMOTIONS, JOY, ANTICIPATION CRITICAL","- Hyvä että JOKU näkee uuden , tyhmän veroperu...","Hyvä että JOKU näkee uuden, tyhmän veroperuste...",Meria,"- Hyvä että JOKU näkee uuden , tyhmän veroperu...","Hyvä että JOKU näkee uuden, tyhmän veroperuste...",124,"- hyvä että joku nähdä uusi , tyhmä veroperust...","hyvä että joku nähdä uusi , tyhmä veroperuste ..."
8,9,- Kukaan luterilainen ei korota itseään hänen ...,2,-2,False,0,NaN,NaN,NaN,SADNESS,- Kukaan luterilainen ei korota itseään hänen ...,Kukaan luterilainen ei korota itseään hänen su...,Meria,- Kukaan luterilainen ei korota itseään hänen ...,Kukaan luterilainen ei korota itseään hänen su...,105,- kukaan luterilainen ei korottaa itse hän suh...,kukaan luterilainen ei korottaa itse hän suhde...
9,10,- kuten on sovittu -,0,0,False,0,NaN,NaN,NaN,NEUTRAL NONE,- kuten on sovittu -,kuten on sovittu,Meria,- kuten on sovittu -,kuten on sovittu,16,- kuten olla sopia -,kuten olla sopia


## Nxgram frequency analysis for 'raw' and 'corrected' HS 
###  - cleaned,
### - lower caps, 
### - but stopwords (or sentiment / emotion words) NOT removed !!!

In [14]:
# import data from xlsx file to pd dataFolder
cwd = os.getcwd()
folder = '\\data\\'
#csv_file = 'HSTrainData.xlsx'
csv_file = 'Manually_Annotated_Collection_ALL_FINAL_22023Merja.xlsx'
csv_source = cwd + folder + csv_file
#print(csv_source)
df = pd.read_excel(csv_source)
df

,id,sample,sentiment,polarity,HSbinary,HSstrength,HStarget,HStopic,HSform,emotion,urban_finnish,correct_finnish,user_nick
0,1,- Ajatus siitä että kaikki henkilön tienaamat...,negative,-1,False,0,NaN,NaN,NaN,"UNPLEASENT, ANTICIPATION CRITICAL, SARCASTIC",- Ajatus siitä että kaikki henkilön tienaamat...,- Ajatus siitä että kaikki henkilön tienaamat...,Meria
1,2,"- Kaivovertauksessa, viime hallitus on kaivan...",negative,-2,False,0,NaN,NaN,NaN,"NEUTRAL NONE, UNPLEASENT, ANTICIPATION CRITICAL","- Kaivovertauksessa, viime hallitus on kaivan...","- Kaivovertauksessa, viime hallitus on kaivan...",Meria
2,3,--´973´¤-.ttu,negative,-5,True,2,NONE,OTHER,"SWEARING, GRANULATED",CONTEMPT DISRESPECT,--´973´¤-.ttu,973 vittu,Meria
3,4,-=>Widdu joo<=-,negative,-4,True,2,NONE,TROLLING,"JOKE SARCASM, SWEARING, GRANULATED",SARCASTIC,-=> Widdu joo <=-,-=> vittu joo <=-,Meria
4,5,"- Ei , mutta olen joutunut elämään katsomalla ...",negative,-5,False,0,NaN,NaN,NaN,"SADNESS, FEAR","- Ei , mutta olen joutunut elämään katsomalla ...","Ei , mutta olen joutunut elämään katsomalla ku...",Meria
...,...,...,...,...,...,...,...,...,...,...,...,...,...
6624,6625,😂😂😂😂😂😂😂😂😂😂😂😂😂😂,positive,5,False,0,NaN,NaN,NaN,JOY,😂😂😂😂😂😂😂😂😂😂😂😂😂😂,😂😂😂😂😂,Meria
6625,6626,😠👉👩‍💼,negative,-3,False,0,NaN,NaN,NaN,ANGER HATE,😠👉👩‍💼,😠👉👩‍💼,Meria
6626,6627,🤣🤣🤣🤣🤣🤣🤣🤣🤣🤣,positive,5,False,0,NaN,NaN,NaN,JOY,🤣🤣🤣🤣🤣🤣🤣🤣🤣🤣,🤣🤣🤣🤣🤣🤣🤣🤣🤣🤣,Meria
6627,6628,🤮😡,negative,-4,False,0,NaN,NaN,NaN,"DISGUST, ANGER HATE",🤮😡,🤮😡,Meria


In [15]:
# Change text samples to string
df["stringSample"] = df[str("sample")]
df["stringCorrect"] = df[str("correct_finnish")]

# add string len column
df["stringLength"] = ""

# extract string length measures of data
def extract_stringLen(df):
    for item in df["stringCorrect"]:
        #newframe["stringLength"] = len(str(item))
        df["stringLength"] = df.stringCorrect.str.len()
    return df

extract_stringLen(df)
df.head(50)

,id,sample,sentiment,polarity,HSbinary,HSstrength,HStarget,HStopic,HSform,emotion,urban_finnish,correct_finnish,user_nick,stringSample,stringCorrect,stringLength
0,1,- Ajatus siitä että kaikki henkilön tienaamat...,negative,-1,False,0,NaN,NaN,NaN,"UNPLEASENT, ANTICIPATION CRITICAL, SARCASTIC",- Ajatus siitä että kaikki henkilön tienaamat...,- Ajatus siitä että kaikki henkilön tienaamat...,Meria,- Ajatus siitä että kaikki henkilön tienaamat...,- Ajatus siitä että kaikki henkilön tienaamat...,866
1,2,"- Kaivovertauksessa, viime hallitus on kaivan...",negative,-2,False,0,NaN,NaN,NaN,"NEUTRAL NONE, UNPLEASENT, ANTICIPATION CRITICAL","- Kaivovertauksessa, viime hallitus on kaivan...","- Kaivovertauksessa, viime hallitus on kaivan...",Meria,"- Kaivovertauksessa, viime hallitus on kaivan...","- Kaivovertauksessa, viime hallitus on kaivan...",984
2,3,--´973´¤-.ttu,negative,-5,True,2,NONE,OTHER,"SWEARING, GRANULATED",CONTEMPT DISRESPECT,--´973´¤-.ttu,973 vittu,Meria,--´973´¤-.ttu,973 vittu,9
3,4,-=>Widdu joo<=-,negative,-4,True,2,NONE,TROLLING,"JOKE SARCASM, SWEARING, GRANULATED",SARCASTIC,-=> Widdu joo <=-,-=> vittu joo <=-,Meria,-=>Widdu joo<=-,-=> vittu joo <=-,17
4,5,"- Ei , mutta olen joutunut elämään katsomalla ...",negative,-5,False,0,NaN,NaN,NaN,"SADNESS, FEAR","- Ei , mutta olen joutunut elämään katsomalla ...","Ei , mutta olen joutunut elämään katsomalla ku...",Meria,"- Ei , mutta olen joutunut elämään katsomalla ...","Ei , mutta olen joutunut elämään katsomalla ku...",144
5,6,- ei viddu -,negative,-5,True,2,NONE,"OPINION, TROLLING","SWEARING, GRANULATED",SURPRISE,ei viddu,ei vittu,Meria,- ei viddu -,ei vittu,8
6,7,- Huvittunut -,positive,1,False,0,NaN,NaN,NaN,JOY,Huvittunut,huvittunut,Meria,- Huvittunut -,huvittunut,10
7,8,"- Hyvä että JOKU näkee uuden , tyhmän veroperu...",negative,-4,True,1,NONE,OPINION,SWEARING,"MIXED EMOTIONS, JOY, ANTICIPATION CRITICAL","- Hyvä että JOKU näkee uuden , tyhmän veroperu...","Hyvä että JOKU näkee uuden, tyhmän veroperuste...",Meria,"- Hyvä että JOKU näkee uuden , tyhmän veroperu...","Hyvä että JOKU näkee uuden, tyhmän veroperuste...",124
8,9,- Kukaan luterilainen ei korota itseään hänen ...,negative,-2,False,0,NaN,NaN,NaN,SADNESS,- Kukaan luterilainen ei korota itseään hänen ...,Kukaan luterilainen ei korota itseään hänen su...,Meria,- Kukaan luterilainen ei korota itseään hänen ...,Kukaan luterilainen ei korota itseään hänen su...,105
9,10,- kuten on sovittu -,neutral,0,False,0,NaN,NaN,NaN,NEUTRAL NONE,- kuten on sovittu -,kuten on sovittu,Meria,- kuten on sovittu -,kuten on sovittu,16


In [16]:
# change sentiment values
df['sentiment'] = df['sentiment'].replace('neutral', 0)
df['sentiment'] = df['sentiment'].replace('positive', 1)
df['sentiment'] = df['sentiment'].replace('negative', 2)

# change HS / not HS values
df['HSbinary'] = df['HSbinary'].replace(False, 0)
df['HSbinary'] = df['HSbinary'].replace(True, 1)
df

,id,sample,sentiment,polarity,HSbinary,HSstrength,HStarget,HStopic,HSform,emotion,urban_finnish,correct_finnish,user_nick,stringSample,stringCorrect,stringLength
0,1,- Ajatus siitä että kaikki henkilön tienaamat...,2,-1,0,0,NaN,NaN,NaN,"UNPLEASENT, ANTICIPATION CRITICAL, SARCASTIC",- Ajatus siitä että kaikki henkilön tienaamat...,- Ajatus siitä että kaikki henkilön tienaamat...,Meria,- Ajatus siitä että kaikki henkilön tienaamat...,- Ajatus siitä että kaikki henkilön tienaamat...,866
1,2,"- Kaivovertauksessa, viime hallitus on kaivan...",2,-2,0,0,NaN,NaN,NaN,"NEUTRAL NONE, UNPLEASENT, ANTICIPATION CRITICAL","- Kaivovertauksessa, viime hallitus on kaivan...","- Kaivovertauksessa, viime hallitus on kaivan...",Meria,"- Kaivovertauksessa, viime hallitus on kaivan...","- Kaivovertauksessa, viime hallitus on kaivan...",984
2,3,--´973´¤-.ttu,2,-5,1,2,NONE,OTHER,"SWEARING, GRANULATED",CONTEMPT DISRESPECT,--´973´¤-.ttu,973 vittu,Meria,--´973´¤-.ttu,973 vittu,9
3,4,-=>Widdu joo<=-,2,-4,1,2,NONE,TROLLING,"JOKE SARCASM, SWEARING, GRANULATED",SARCASTIC,-=> Widdu joo <=-,-=> vittu joo <=-,Meria,-=>Widdu joo<=-,-=> vittu joo <=-,17
4,5,"- Ei , mutta olen joutunut elämään katsomalla ...",2,-5,0,0,NaN,NaN,NaN,"SADNESS, FEAR","- Ei , mutta olen joutunut elämään katsomalla ...","Ei , mutta olen joutunut elämään katsomalla ku...",Meria,"- Ei , mutta olen joutunut elämään katsomalla ...","Ei , mutta olen joutunut elämään katsomalla ku...",144
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6624,6625,😂😂😂😂😂😂😂😂😂😂😂😂😂😂,1,5,0,0,NaN,NaN,NaN,JOY,😂😂😂😂😂😂😂😂😂😂😂😂😂😂,😂😂😂😂😂,Meria,😂😂😂😂😂😂😂😂😂😂😂😂😂😂,😂😂😂😂😂,5
6625,6626,😠👉👩‍💼,2,-3,0,0,NaN,NaN,NaN,ANGER HATE,😠👉👩‍💼,😠👉👩‍💼,Meria,😠👉👩‍💼,😠👉👩‍💼,5
6626,6627,🤣🤣🤣🤣🤣🤣🤣🤣🤣🤣,1,5,0,0,NaN,NaN,NaN,JOY,🤣🤣🤣🤣🤣🤣🤣🤣🤣🤣,🤣🤣🤣🤣🤣🤣🤣🤣🤣🤣,Meria,🤣🤣🤣🤣🤣🤣🤣🤣🤣🤣,🤣🤣🤣🤣🤣🤣🤣🤣🤣🤣,10
6627,6628,🤮😡,2,-4,0,0,NaN,NaN,NaN,"DISGUST, ANGER HATE",🤮😡,🤮😡,Meria,🤮😡,🤮😡,2


In [17]:
# Lemmatization of raw samples
df['sampleLemmas'] = df['stringSample'].apply(lambda row: " ".join([token.lemma_ for token in FinSpacyExp(row)]))

# Lemmatization of corrected samples
df['correctLemmas'] = df['stringCorrect'].apply(lambda row: " ".join([token.lemma_ for token in FinSpacyExp(row)]))
df.head(10)

,id,sample,sentiment,polarity,HSbinary,HSstrength,HStarget,HStopic,HSform,emotion,urban_finnish,correct_finnish,user_nick,stringSample,stringCorrect,stringLength,sampleLemmas,correctLemmas
0,1,- Ajatus siitä että kaikki henkilön tienaamat...,2,-1,0,0,NaN,NaN,NaN,"UNPLEASENT, ANTICIPATION CRITICAL, SARCASTIC",- Ajatus siitä että kaikki henkilön tienaamat...,- Ajatus siitä että kaikki henkilön tienaamat...,Meria,- Ajatus siitä että kaikki henkilön tienaamat...,- Ajatus siitä että kaikki henkilön tienaamat...,866,- ajatus se että kaikki henkilö tienata tulo...,- ajatus se että kaikki henkilö tienata tulo...
1,2,"- Kaivovertauksessa, viime hallitus on kaivan...",2,-2,0,0,NaN,NaN,NaN,"NEUTRAL NONE, UNPLEASENT, ANTICIPATION CRITICAL","- Kaivovertauksessa, viime hallitus on kaivan...","- Kaivovertauksessa, viime hallitus on kaivan...",Meria,"- Kaivovertauksessa, viime hallitus on kaivan...","- Kaivovertauksessa, viime hallitus on kaivan...",984,"- kaivovertaus , viime hallitus olla kaivaa ...","- kaivovertaus , viime hallitus olla kaivaa ..."
2,3,--´973´¤-.ttu,2,-5,1,2,NONE,OTHER,"SWEARING, GRANULATED",CONTEMPT DISRESPECT,--´973´¤-.ttu,973 vittu,Meria,--´973´¤-.ttu,973 vittu,9,--´973´¤-.ttu,973 vittu
3,4,-=>Widdu joo<=-,2,-4,1,2,NONE,TROLLING,"JOKE SARCASM, SWEARING, GRANULATED",SARCASTIC,-=> Widdu joo <=-,-=> vittu joo <=-,Meria,-=>Widdu joo<=-,-=> vittu joo <=-,17,-=>Widdu joo<= -,-= > vittu joo < = -
4,5,"- Ei , mutta olen joutunut elämään katsomalla ...",2,-5,0,0,NaN,NaN,NaN,"SADNESS, FEAR","- Ei , mutta olen joutunut elämään katsomalla ...","Ei , mutta olen joutunut elämään katsomalla ku...",Meria,"- Ei , mutta olen joutunut elämään katsomalla ...","Ei , mutta olen joutunut elämään katsomalla ku...",144,"- ei , mutta olla joutua elää katsoa kuinka ki...","ei , mutta olla joutua elää katsoa kuinka kius..."
5,6,- ei viddu -,2,-5,1,2,NONE,"OPINION, TROLLING","SWEARING, GRANULATED",SURPRISE,ei viddu,ei vittu,Meria,- ei viddu -,ei vittu,8,- ei viddu -,ei vittu
6,7,- Huvittunut -,1,1,0,0,NaN,NaN,NaN,JOY,Huvittunut,huvittunut,Meria,- Huvittunut -,huvittunut,10,- huvittunut -,huvittua
7,8,"- Hyvä että JOKU näkee uuden , tyhmän veroperu...",2,-4,1,1,NONE,OPINION,SWEARING,"MIXED EMOTIONS, JOY, ANTICIPATION CRITICAL","- Hyvä että JOKU näkee uuden , tyhmän veroperu...","Hyvä että JOKU näkee uuden, tyhmän veroperuste...",Meria,"- Hyvä että JOKU näkee uuden , tyhmän veroperu...","Hyvä että JOKU näkee uuden, tyhmän veroperuste...",124,"- hyvä että joku nähdä uusi , tyhmä veroperust...","hyvä että joku nähdä uusi , tyhmä veroperuste ..."
8,9,- Kukaan luterilainen ei korota itseään hänen ...,2,-2,0,0,NaN,NaN,NaN,SADNESS,- Kukaan luterilainen ei korota itseään hänen ...,Kukaan luterilainen ei korota itseään hänen su...,Meria,- Kukaan luterilainen ei korota itseään hänen ...,Kukaan luterilainen ei korota itseään hänen su...,105,- kukaan luterilainen ei korottaa itse hän suh...,kukaan luterilainen ei korottaa itse hän suhde...
9,10,- kuten on sovittu -,0,0,0,0,NaN,NaN,NaN,NEUTRAL NONE,- kuten on sovittu -,kuten on sovittu,Meria,- kuten on sovittu -,kuten on sovittu,16,- kuten olla sopia -,kuten olla sopia


In [18]:
# Tokenization of raw samples with NLTK casual tokenization
df['sampleTokens'] = df['sampleLemmas'].apply(lambda row: casual_tokenize(row))

# Tokenization of corrected samples with NLTK casual tokenization
df['correctTokens'] = df['correctLemmas'].apply(lambda row: casual_tokenize(row))
df.head(10)

,id,sample,sentiment,polarity,HSbinary,HSstrength,HStarget,HStopic,HSform,emotion,urban_finnish,correct_finnish,user_nick,stringSample,stringCorrect,stringLength,sampleLemmas,correctLemmas,sampleTokens,correctTokens
0,1,- Ajatus siitä että kaikki henkilön tienaamat...,2,-1,0,0,NaN,NaN,NaN,"UNPLEASENT, ANTICIPATION CRITICAL, SARCASTIC",- Ajatus siitä että kaikki henkilön tienaamat...,- Ajatus siitä että kaikki henkilön tienaamat...,Meria,- Ajatus siitä että kaikki henkilön tienaamat...,- Ajatus siitä että kaikki henkilön tienaamat...,866,- ajatus se että kaikki henkilö tienata tulo...,- ajatus se että kaikki henkilö tienata tulo...,"[-, ajatus, se, että, kaikki, henkilö, tienata...","[-, ajatus, se, että, kaikki, henkilö, tienata..."
1,2,"- Kaivovertauksessa, viime hallitus on kaivan...",2,-2,0,0,NaN,NaN,NaN,"NEUTRAL NONE, UNPLEASENT, ANTICIPATION CRITICAL","- Kaivovertauksessa, viime hallitus on kaivan...","- Kaivovertauksessa, viime hallitus on kaivan...",Meria,"- Kaivovertauksessa, viime hallitus on kaivan...","- Kaivovertauksessa, viime hallitus on kaivan...",984,"- kaivovertaus , viime hallitus olla kaivaa ...","- kaivovertaus , viime hallitus olla kaivaa ...","[-, kaivovertaus, ,, viime, hallitus, olla, ka...","[-, kaivovertaus, ,, viime, hallitus, olla, ka..."
2,3,--´973´¤-.ttu,2,-5,1,2,NONE,OTHER,"SWEARING, GRANULATED",CONTEMPT DISRESPECT,--´973´¤-.ttu,973 vittu,Meria,--´973´¤-.ttu,973 vittu,9,--´973´¤-.ttu,973 vittu,"[-, -, ´, 973, ´, ¤, -, ., ttu]","[973, vittu]"
3,4,-=>Widdu joo<=-,2,-4,1,2,NONE,TROLLING,"JOKE SARCASM, SWEARING, GRANULATED",SARCASTIC,-=> Widdu joo <=-,-=> vittu joo <=-,Meria,-=>Widdu joo<=-,-=> vittu joo <=-,17,-=>Widdu joo<= -,-= > vittu joo < = -,"[-, =, >, Widdu, joo, <, =, -]","[-, =, >, vittu, joo, <, =, -]"
4,5,"- Ei , mutta olen joutunut elämään katsomalla ...",2,-5,0,0,NaN,NaN,NaN,"SADNESS, FEAR","- Ei , mutta olen joutunut elämään katsomalla ...","Ei , mutta olen joutunut elämään katsomalla ku...",Meria,"- Ei , mutta olen joutunut elämään katsomalla ...","Ei , mutta olen joutunut elämään katsomalla ku...",144,"- ei , mutta olla joutua elää katsoa kuinka ki...","ei , mutta olla joutua elää katsoa kuinka kius...","[-, ei, ,, mutta, olla, joutua, elää, katsoa, ...","[ei, ,, mutta, olla, joutua, elää, katsoa, kui..."
5,6,- ei viddu -,2,-5,1,2,NONE,"OPINION, TROLLING","SWEARING, GRANULATED",SURPRISE,ei viddu,ei vittu,Meria,- ei viddu -,ei vittu,8,- ei viddu -,ei vittu,"[-, ei, viddu, -]","[ei, vittu]"
6,7,- Huvittunut -,1,1,0,0,NaN,NaN,NaN,JOY,Huvittunut,huvittunut,Meria,- Huvittunut -,huvittunut,10,- huvittunut -,huvittua,"[-, huvittunut, -]",[huvittua]
7,8,"- Hyvä että JOKU näkee uuden , tyhmän veroperu...",2,-4,1,1,NONE,OPINION,SWEARING,"MIXED EMOTIONS, JOY, ANTICIPATION CRITICAL","- Hyvä että JOKU näkee uuden , tyhmän veroperu...","Hyvä että JOKU näkee uuden, tyhmän veroperuste...",Meria,"- Hyvä että JOKU näkee uuden , tyhmän veroperu...","Hyvä että JOKU näkee uuden, tyhmän veroperuste...",124,"- hyvä että joku nähdä uusi , tyhmä veroperust...","hyvä että joku nähdä uusi , tyhmä veroperuste ...","[-, hyvä, että, joku, nähdä, uusi, ,, tyhmä, v...","[hyvä, että, joku, nähdä, uusi, ,, tyhmä, vero..."
8,9,- Kukaan luterilainen ei korota itseään hänen ...,2,-2,0,0,NaN,NaN,NaN,SADNESS,- Kukaan luterilainen ei korota itseään hänen ...,Kukaan luterilainen ei korota itseään hänen su...,Meria,- Kukaan luterilainen ei korota itseään hänen ...,Kukaan luterilainen ei korota itseään hänen su...,105,- kukaan luterilainen ei korottaa itse hän suh...,kukaan luterilainen ei korottaa itse hän suhde...,"[-, kukaan, luterilainen, ei, korottaa, itse, ...","[kukaan, luterilainen, ei, korottaa, itse, hän..."
9,10,- kuten on sovittu -,0,0,0,0,NaN,NaN,NaN,NEUTRAL NONE,- kuten on sovittu -,kuten on sovittu,Meria,- kuten on sovittu -,kuten on sovittu,16,- kuten olla sopia -,kuten olla sopia,"[-, kuten, olla, sopia, -]","[kuten, olla, sopia]"


In [19]:
# Change characters to lower chars
df['sampleTokens'] = df['sampleTokens'].apply(lambda row: [token.lower() for token in row])
df['correctTokens'] = df['correctTokens'].apply(lambda row: [token.lower() for token in row])
df.head(10)

,id,sample,sentiment,polarity,HSbinary,HSstrength,HStarget,HStopic,HSform,emotion,urban_finnish,correct_finnish,user_nick,stringSample,stringCorrect,stringLength,sampleLemmas,correctLemmas,sampleTokens,correctTokens
0,1,- Ajatus siitä että kaikki henkilön tienaamat...,2,-1,0,0,NaN,NaN,NaN,"UNPLEASENT, ANTICIPATION CRITICAL, SARCASTIC",- Ajatus siitä että kaikki henkilön tienaamat...,- Ajatus siitä että kaikki henkilön tienaamat...,Meria,- Ajatus siitä että kaikki henkilön tienaamat...,- Ajatus siitä että kaikki henkilön tienaamat...,866,- ajatus se että kaikki henkilö tienata tulo...,- ajatus se että kaikki henkilö tienata tulo...,"[-, ajatus, se, että, kaikki, henkilö, tienata...","[-, ajatus, se, että, kaikki, henkilö, tienata..."
1,2,"- Kaivovertauksessa, viime hallitus on kaivan...",2,-2,0,0,NaN,NaN,NaN,"NEUTRAL NONE, UNPLEASENT, ANTICIPATION CRITICAL","- Kaivovertauksessa, viime hallitus on kaivan...","- Kaivovertauksessa, viime hallitus on kaivan...",Meria,"- Kaivovertauksessa, viime hallitus on kaivan...","- Kaivovertauksessa, viime hallitus on kaivan...",984,"- kaivovertaus , viime hallitus olla kaivaa ...","- kaivovertaus , viime hallitus olla kaivaa ...","[-, kaivovertaus, ,, viime, hallitus, olla, ka...","[-, kaivovertaus, ,, viime, hallitus, olla, ka..."
2,3,--´973´¤-.ttu,2,-5,1,2,NONE,OTHER,"SWEARING, GRANULATED",CONTEMPT DISRESPECT,--´973´¤-.ttu,973 vittu,Meria,--´973´¤-.ttu,973 vittu,9,--´973´¤-.ttu,973 vittu,"[-, -, ´, 973, ´, ¤, -, ., ttu]","[973, vittu]"
3,4,-=>Widdu joo<=-,2,-4,1,2,NONE,TROLLING,"JOKE SARCASM, SWEARING, GRANULATED",SARCASTIC,-=> Widdu joo <=-,-=> vittu joo <=-,Meria,-=>Widdu joo<=-,-=> vittu joo <=-,17,-=>Widdu joo<= -,-= > vittu joo < = -,"[-, =, >, widdu, joo, <, =, -]","[-, =, >, vittu, joo, <, =, -]"
4,5,"- Ei , mutta olen joutunut elämään katsomalla ...",2,-5,0,0,NaN,NaN,NaN,"SADNESS, FEAR","- Ei , mutta olen joutunut elämään katsomalla ...","Ei , mutta olen joutunut elämään katsomalla ku...",Meria,"- Ei , mutta olen joutunut elämään katsomalla ...","Ei , mutta olen joutunut elämään katsomalla ku...",144,"- ei , mutta olla joutua elää katsoa kuinka ki...","ei , mutta olla joutua elää katsoa kuinka kius...","[-, ei, ,, mutta, olla, joutua, elää, katsoa, ...","[ei, ,, mutta, olla, joutua, elää, katsoa, kui..."
5,6,- ei viddu -,2,-5,1,2,NONE,"OPINION, TROLLING","SWEARING, GRANULATED",SURPRISE,ei viddu,ei vittu,Meria,- ei viddu -,ei vittu,8,- ei viddu -,ei vittu,"[-, ei, viddu, -]","[ei, vittu]"
6,7,- Huvittunut -,1,1,0,0,NaN,NaN,NaN,JOY,Huvittunut,huvittunut,Meria,- Huvittunut -,huvittunut,10,- huvittunut -,huvittua,"[-, huvittunut, -]",[huvittua]
7,8,"- Hyvä että JOKU näkee uuden , tyhmän veroperu...",2,-4,1,1,NONE,OPINION,SWEARING,"MIXED EMOTIONS, JOY, ANTICIPATION CRITICAL","- Hyvä että JOKU näkee uuden , tyhmän veroperu...","Hyvä että JOKU näkee uuden, tyhmän veroperuste...",Meria,"- Hyvä että JOKU näkee uuden , tyhmän veroperu...","Hyvä että JOKU näkee uuden, tyhmän veroperuste...",124,"- hyvä että joku nähdä uusi , tyhmä veroperust...","hyvä että joku nähdä uusi , tyhmä veroperuste ...","[-, hyvä, että, joku, nähdä, uusi, ,, tyhmä, v...","[hyvä, että, joku, nähdä, uusi, ,, tyhmä, vero..."
8,9,- Kukaan luterilainen ei korota itseään hänen ...,2,-2,0,0,NaN,NaN,NaN,SADNESS,- Kukaan luterilainen ei korota itseään hänen ...,Kukaan luterilainen ei korota itseään hänen su...,Meria,- Kukaan luterilainen ei korota itseään hänen ...,Kukaan luterilainen ei korota itseään hänen su...,105,- kukaan luterilainen ei korottaa itse hän suh...,kukaan luterilainen ei korottaa itse hän suhde...,"[-, kukaan, luterilainen, ei, korottaa, itse, ...","[kukaan, luterilainen, ei, korottaa, itse, hän..."
9,10,- kuten on sovittu -,0,0,0,0,NaN,NaN,NaN,NEUTRAL NONE,- kuten on sovittu -,kuten on sovittu,Meria,- kuten on sovittu -,kuten on sovittu,16,- kuten olla sopia -,kuten olla sopia,"[-, kuten, olla, sopia, -]","[kuten, olla, sopia]"


In [20]:
# cleaning - remove all non alphabets
regex = re.compile('[\W]+')

# Clean raw samples
df['sampleTokens'] = df['sampleTokens'].apply(lambda row: [regex.sub(' ', token) for token in row])
# Clean corrected samples
df['correctTokens'] = df['correctTokens'].apply(lambda row: [regex.sub(' ', token) for token in row])
df.head(10)

,id,sample,sentiment,polarity,HSbinary,HSstrength,HStarget,HStopic,HSform,emotion,urban_finnish,correct_finnish,user_nick,stringSample,stringCorrect,stringLength,sampleLemmas,correctLemmas,sampleTokens,correctTokens
0,1,- Ajatus siitä että kaikki henkilön tienaamat...,2,-1,0,0,NaN,NaN,NaN,"UNPLEASENT, ANTICIPATION CRITICAL, SARCASTIC",- Ajatus siitä että kaikki henkilön tienaamat...,- Ajatus siitä että kaikki henkilön tienaamat...,Meria,- Ajatus siitä että kaikki henkilön tienaamat...,- Ajatus siitä että kaikki henkilön tienaamat...,866,- ajatus se että kaikki henkilö tienata tulo...,- ajatus se että kaikki henkilö tienata tulo...,"[ , ajatus, se, että, kaikki, henkilö, tienata...","[ , ajatus, se, että, kaikki, henkilö, tienata..."
1,2,"- Kaivovertauksessa, viime hallitus on kaivan...",2,-2,0,0,NaN,NaN,NaN,"NEUTRAL NONE, UNPLEASENT, ANTICIPATION CRITICAL","- Kaivovertauksessa, viime hallitus on kaivan...","- Kaivovertauksessa, viime hallitus on kaivan...",Meria,"- Kaivovertauksessa, viime hallitus on kaivan...","- Kaivovertauksessa, viime hallitus on kaivan...",984,"- kaivovertaus , viime hallitus olla kaivaa ...","- kaivovertaus , viime hallitus olla kaivaa ...","[ , kaivovertaus, , viime, hallitus, olla, ka...","[ , kaivovertaus, , viime, hallitus, olla, ka..."
2,3,--´973´¤-.ttu,2,-5,1,2,NONE,OTHER,"SWEARING, GRANULATED",CONTEMPT DISRESPECT,--´973´¤-.ttu,973 vittu,Meria,--´973´¤-.ttu,973 vittu,9,--´973´¤-.ttu,973 vittu,"[ , , , 973, , , , , ttu]","[973, vittu]"
3,4,-=>Widdu joo<=-,2,-4,1,2,NONE,TROLLING,"JOKE SARCASM, SWEARING, GRANULATED",SARCASTIC,-=> Widdu joo <=-,-=> vittu joo <=-,Meria,-=>Widdu joo<=-,-=> vittu joo <=-,17,-=>Widdu joo<= -,-= > vittu joo < = -,"[ , , , widdu, joo, , , ]","[ , , , vittu, joo, , , ]"
4,5,"- Ei , mutta olen joutunut elämään katsomalla ...",2,-5,0,0,NaN,NaN,NaN,"SADNESS, FEAR","- Ei , mutta olen joutunut elämään katsomalla ...","Ei , mutta olen joutunut elämään katsomalla ku...",Meria,"- Ei , mutta olen joutunut elämään katsomalla ...","Ei , mutta olen joutunut elämään katsomalla ku...",144,"- ei , mutta olla joutua elää katsoa kuinka ki...","ei , mutta olla joutua elää katsoa kuinka kius...","[ , ei, , mutta, olla, joutua, elää, katsoa, ...","[ei, , mutta, olla, joutua, elää, katsoa, kui..."
5,6,- ei viddu -,2,-5,1,2,NONE,"OPINION, TROLLING","SWEARING, GRANULATED",SURPRISE,ei viddu,ei vittu,Meria,- ei viddu -,ei vittu,8,- ei viddu -,ei vittu,"[ , ei, viddu, ]","[ei, vittu]"
6,7,- Huvittunut -,1,1,0,0,NaN,NaN,NaN,JOY,Huvittunut,huvittunut,Meria,- Huvittunut -,huvittunut,10,- huvittunut -,huvittua,"[ , huvittunut, ]",[huvittua]
7,8,"- Hyvä että JOKU näkee uuden , tyhmän veroperu...",2,-4,1,1,NONE,OPINION,SWEARING,"MIXED EMOTIONS, JOY, ANTICIPATION CRITICAL","- Hyvä että JOKU näkee uuden , tyhmän veroperu...","Hyvä että JOKU näkee uuden, tyhmän veroperuste...",Meria,"- Hyvä että JOKU näkee uuden , tyhmän veroperu...","Hyvä että JOKU näkee uuden, tyhmän veroperuste...",124,"- hyvä että joku nähdä uusi , tyhmä veroperust...","hyvä että joku nähdä uusi , tyhmä veroperuste ...","[ , hyvä, että, joku, nähdä, uusi, , tyhmä, v...","[hyvä, että, joku, nähdä, uusi, , tyhmä, vero..."
8,9,- Kukaan luterilainen ei korota itseään hänen ...,2,-2,0,0,NaN,NaN,NaN,SADNESS,- Kukaan luterilainen ei korota itseään hänen ...,Kukaan luterilainen ei korota itseään hänen su...,Meria,- Kukaan luterilainen ei korota itseään hänen ...,Kukaan luterilainen ei korota itseään hänen su...,105,- kukaan luterilainen ei korottaa itse hän suh...,kukaan luterilainen ei korottaa itse hän suhde...,"[ , kukaan, luterilainen, ei, korottaa, itse, ...","[kukaan, luterilainen, ei, korottaa, itse, hän..."
9,10,- kuten on sovittu -,0,0,0,0,NaN,NaN,NaN,NEUTRAL NONE,- kuten on sovittu -,kuten on sovittu,Meria,- kuten on sovittu -,kuten on sovittu,16,- kuten olla sopia -,kuten olla sopia,"[ , kuten, olla, sopia, ]","[kuten, olla, sopia]"


In [21]:
# cleaning - remove all non alphabets

# Clean raw samples
df['sampleTokens'] = df['sampleTokens'].apply(lambda row: ' '.join([token for token in row if token.isalpha()]))
# Clean corrected samples
df['correctTokens'] = df['correctTokens'].apply(lambda row: ' '.join([token for token in row if token.isalpha()]))
df.head(10)

,id,sample,sentiment,polarity,HSbinary,HSstrength,HStarget,HStopic,HSform,emotion,urban_finnish,correct_finnish,user_nick,stringSample,stringCorrect,stringLength,sampleLemmas,correctLemmas,sampleTokens,correctTokens
0,1,- Ajatus siitä että kaikki henkilön tienaamat...,2,-1,0,0,NaN,NaN,NaN,"UNPLEASENT, ANTICIPATION CRITICAL, SARCASTIC",- Ajatus siitä että kaikki henkilön tienaamat...,- Ajatus siitä että kaikki henkilön tienaamat...,Meria,- Ajatus siitä että kaikki henkilön tienaamat...,- Ajatus siitä että kaikki henkilön tienaamat...,866,- ajatus se että kaikki henkilö tienata tulo...,- ajatus se että kaikki henkilö tienata tulo...,ajatus se että kaikki henkilö tienata tulo oll...,ajatus se että kaikki henkilö tienata tulo oll...
1,2,"- Kaivovertauksessa, viime hallitus on kaivan...",2,-2,0,0,NaN,NaN,NaN,"NEUTRAL NONE, UNPLEASENT, ANTICIPATION CRITICAL","- Kaivovertauksessa, viime hallitus on kaivan...","- Kaivovertauksessa, viime hallitus on kaivan...",Meria,"- Kaivovertauksessa, viime hallitus on kaivan...","- Kaivovertauksessa, viime hallitus on kaivan...",984,"- kaivovertaus , viime hallitus olla kaivaa ...","- kaivovertaus , viime hallitus olla kaivaa ...",kaivovertaus viime hallitus olla kaivaa se kai...,kaivovertaus viime hallitus olla kaivaa se kai...
2,3,--´973´¤-.ttu,2,-5,1,2,NONE,OTHER,"SWEARING, GRANULATED",CONTEMPT DISRESPECT,--´973´¤-.ttu,973 vittu,Meria,--´973´¤-.ttu,973 vittu,9,--´973´¤-.ttu,973 vittu,ttu,vittu
3,4,-=>Widdu joo<=-,2,-4,1,2,NONE,TROLLING,"JOKE SARCASM, SWEARING, GRANULATED",SARCASTIC,-=> Widdu joo <=-,-=> vittu joo <=-,Meria,-=>Widdu joo<=-,-=> vittu joo <=-,17,-=>Widdu joo<= -,-= > vittu joo < = -,widdu joo,vittu joo
4,5,"- Ei , mutta olen joutunut elämään katsomalla ...",2,-5,0,0,NaN,NaN,NaN,"SADNESS, FEAR","- Ei , mutta olen joutunut elämään katsomalla ...","Ei , mutta olen joutunut elämään katsomalla ku...",Meria,"- Ei , mutta olen joutunut elämään katsomalla ...","Ei , mutta olen joutunut elämään katsomalla ku...",144,"- ei , mutta olla joutua elää katsoa kuinka ki...","ei , mutta olla joutua elää katsoa kuinka kius...",ei mutta olla joutua elää katsoa kuinka kiusat...,ei mutta olla joutua elää katsoa kuinka kiusat...
5,6,- ei viddu -,2,-5,1,2,NONE,"OPINION, TROLLING","SWEARING, GRANULATED",SURPRISE,ei viddu,ei vittu,Meria,- ei viddu -,ei vittu,8,- ei viddu -,ei vittu,ei viddu,ei vittu
6,7,- Huvittunut -,1,1,0,0,NaN,NaN,NaN,JOY,Huvittunut,huvittunut,Meria,- Huvittunut -,huvittunut,10,- huvittunut -,huvittua,huvittunut,huvittua
7,8,"- Hyvä että JOKU näkee uuden , tyhmän veroperu...",2,-4,1,1,NONE,OPINION,SWEARING,"MIXED EMOTIONS, JOY, ANTICIPATION CRITICAL","- Hyvä että JOKU näkee uuden , tyhmän veroperu...","Hyvä että JOKU näkee uuden, tyhmän veroperuste...",Meria,"- Hyvä että JOKU näkee uuden , tyhmän veroperu...","Hyvä että JOKU näkee uuden, tyhmän veroperuste...",124,"- hyvä että joku nähdä uusi , tyhmä veroperust...","hyvä että joku nähdä uusi , tyhmä veroperuste ...",hyvä että joku nähdä uusi tyhmä veroperuste jo...,hyvä että joku nähdä uusi tyhmä veroperuste jo...
8,9,- Kukaan luterilainen ei korota itseään hänen ...,2,-2,0,0,NaN,NaN,NaN,SADNESS,- Kukaan luterilainen ei korota itseään hänen ...,Kukaan luterilainen ei korota itseään hänen su...,Meria,- Kukaan luterilainen ei korota itseään hänen ...,Kukaan luterilainen ei korota itseään hänen su...,105,- kukaan luterilainen ei korottaa itse hän suh...,kukaan luterilainen ei korottaa itse hän suhde...,kukaan luterilainen ei korottaa itse hän suhde...,kukaan luterilainen ei korottaa itse hän suhde...
9,10,- kuten on sovittu -,0,0,0,0,NaN,NaN,NaN,NEUTRAL NONE,- kuten on sovittu -,kuten on sovittu,Meria,- kuten on sovittu -,kuten on sovittu,16,- kuten olla sopia -,kuten olla sopia,kuten olla sopia,kuten olla sopia


## Create sample (row) based nxgrams (2-, 3- and 4-grams) with NLTK ngrams

In [22]:
# create nxgrams with NLTK ngrams

# Nxgrams of raw samples
df['sampleN2grams'] = df['sampleTokens'].str.split().apply(lambda row: list(map(' '.join, ngrams(row, n=2))))
df['sampleN3grams'] = df['sampleTokens'].str.split().apply(lambda row: list(map(' '.join, ngrams(row, n=3))))
df['sampleN4grams'] = df['sampleTokens'].str.split().apply(lambda row: list(map(' '.join, ngrams(row, n=4))))

# Nxgrams of corrected samples
df['correctN2grams'] = df['correctTokens'].str.split().apply(lambda row: list(map(' '.join, ngrams(row, n=2))))
df['correctN3grams'] = df['correctTokens'].str.split().apply(lambda row: list(map(' '.join, ngrams(row, n=3))))
df['correctN4grams'] = df['correctTokens'].str.split().apply(lambda row: list(map(' '.join, ngrams(row, n=4))))
df.head(10)

,id,sample,sentiment,polarity,HSbinary,HSstrength,HStarget,HStopic,HSform,emotion,...,sampleLemmas,correctLemmas,sampleTokens,correctTokens,sampleN2grams,sampleN3grams,sampleN4grams,correctN2grams,correctN3grams,correctN4grams
0,1,- Ajatus siitä että kaikki henkilön tienaamat...,2,-1,0,0,NaN,NaN,NaN,"UNPLEASENT, ANTICIPATION CRITICAL, SARCASTIC",...,- ajatus se että kaikki henkilö tienata tulo...,- ajatus se että kaikki henkilö tienata tulo...,ajatus se että kaikki henkilö tienata tulo oll...,ajatus se että kaikki henkilö tienata tulo oll...,"[ajatus se, se että, että kaikki, kaikki henki...","[ajatus se että, se että kaikki, että kaikki h...","[ajatus se että kaikki, se että kaikki henkilö...","[ajatus se, se että, että kaikki, kaikki henki...","[ajatus se että, se että kaikki, että kaikki h...","[ajatus se että kaikki, se että kaikki henkilö..."
1,2,"- Kaivovertauksessa, viime hallitus on kaivan...",2,-2,0,0,NaN,NaN,NaN,"NEUTRAL NONE, UNPLEASENT, ANTICIPATION CRITICAL",...,"- kaivovertaus , viime hallitus olla kaivaa ...","- kaivovertaus , viime hallitus olla kaivaa ...",kaivovertaus viime hallitus olla kaivaa se kai...,kaivovertaus viime hallitus olla kaivaa se kai...,"[kaivovertaus viime, viime hallitus, hallitus ...","[kaivovertaus viime hallitus, viime hallitus o...","[kaivovertaus viime hallitus olla, viime halli...","[kaivovertaus viime, viime hallitus, hallitus ...","[kaivovertaus viime hallitus, viime hallitus o...","[kaivovertaus viime hallitus olla, viime halli..."
2,3,--´973´¤-.ttu,2,-5,1,2,NONE,OTHER,"SWEARING, GRANULATED",CONTEMPT DISRESPECT,...,--´973´¤-.ttu,973 vittu,ttu,vittu,[],[],[],[],[],[]
3,4,-=>Widdu joo<=-,2,-4,1,2,NONE,TROLLING,"JOKE SARCASM, SWEARING, GRANULATED",SARCASTIC,...,-=>Widdu joo<= -,-= > vittu joo < = -,widdu joo,vittu joo,[widdu joo],[],[],[vittu joo],[],[]
4,5,"- Ei , mutta olen joutunut elämään katsomalla ...",2,-5,0,0,NaN,NaN,NaN,"SADNESS, FEAR",...,"- ei , mutta olla joutua elää katsoa kuinka ki...","ei , mutta olla joutua elää katsoa kuinka kius...",ei mutta olla joutua elää katsoa kuinka kiusat...,ei mutta olla joutua elää katsoa kuinka kiusat...,"[ei mutta, mutta olla, olla joutua, joutua elä...","[ei mutta olla, mutta olla joutua, olla joutua...","[ei mutta olla joutua, mutta olla joutua elää,...","[ei mutta, mutta olla, olla joutua, joutua elä...","[ei mutta olla, mutta olla joutua, olla joutua...","[ei mutta olla joutua, mutta olla joutua elää,..."
5,6,- ei viddu -,2,-5,1,2,NONE,"OPINION, TROLLING","SWEARING, GRANULATED",SURPRISE,...,- ei viddu -,ei vittu,ei viddu,ei vittu,[ei viddu],[],[],[ei vittu],[],[]
6,7,- Huvittunut -,1,1,0,0,NaN,NaN,NaN,JOY,...,- huvittunut -,huvittua,huvittunut,huvittua,[],[],[],[],[],[]
7,8,"- Hyvä että JOKU näkee uuden , tyhmän veroperu...",2,-4,1,1,NONE,OPINION,SWEARING,"MIXED EMOTIONS, JOY, ANTICIPATION CRITICAL",...,"- hyvä että joku nähdä uusi , tyhmä veroperust...","hyvä että joku nähdä uusi , tyhmä veroperuste ...",hyvä että joku nähdä uusi tyhmä veroperuste jo...,hyvä että joku nähdä uusi tyhmä veroperuste jo...,"[hyvä että, että joku, joku nähdä, nähdä uusi,...","[hyvä että joku, että joku nähdä, joku nähdä u...","[hyvä että joku nähdä, että joku nähdä uusi, j...","[hyvä että, että joku, joku nähdä, nähdä uusi,...","[hyvä että joku, että joku nähdä, joku nähdä u...","[hyvä että joku nähdä, että joku nähdä uusi, j..."
8,9,- Kukaan luterilainen ei korota itseään hänen ...,2,-2,0,0,NaN,NaN,NaN,SADNESS,...,- kukaan luterilainen ei korottaa itse hän suh...,kukaan luterilainen ei korottaa itse hän suhde...,kukaan luterilainen ei korottaa itse hän suhde...,kukaan luterilainen ei korottaa itse hän suhde...,"[kukaan luterilainen, luterilainen ei, ei koro...","[kukaan luterilainen ei, luterilainen ei korot...","[kukaan luterilainen ei korottaa, luterilainen...","[kukaan luterilainen, luterilainen ei, ei koro...","[kukaan luterilainen ei, luterilainen ei korot...","[kukaan luterilainen ei korottaa, luterilainen..."
9,10,- kuten on sovittu -,0,0,0,0,NaN,N

## Check row based nxgrams (with print outs)

## bigrams / 2-grams of 'raw' samples

In [23]:
# N2grams of tokenized, lemmatized and cleaned raw samples

all_n2grams = []
for sample in df['sampleN2grams']:
    for item in sample:
        all_n2grams.append(item)
all_n2grams

#count the lemma twograms
N2gram = Counter(all_n2grams)
N2gram

Counter({'ajatus se': 1,
         'se että': 64,
         'että kaikki': 8,
         'kaikki henkilö': 1,
         'henkilö tienata': 1,
         'tienata tulo': 1,
         'tulo olla': 1,
         'olla hän': 10,
         'hän oma': 3,
         'oma työ': 1,
         'työ tulos': 1,
         'tulos olla': 2,
         'olla lähinnä': 1,
         'lähinnä naurettava': 1,
         'naurettava se': 1,
         'se voida': 10,
         'voida ehkä': 2,
         'ehkä päteä': 1,
         'päteä jos': 1,
         'jos henkilö': 1,
         'henkilö elää': 2,
         'elää yksin': 2,
         'yksin metsä': 1,
         'metsä susi': 1,
         'susi kasvattaa': 1,
         'kasvattaa mutta': 1,
         'mutta ei': 24,
         'ei moderni': 1,
         'moderni hyvinvointiyhteiskunta': 1,
         'hyvinvointiyhteiskunta yhteiskunta': 1,
         'yhteiskunta olla': 1,
         'olla maksaa': 2,
         'maksaa lepomäki': 1,
         'lepomäki terveydenhoito': 1,
         'terveydenhoito

In [24]:
# create data frame for lemmas
lemma_freq_N2 = pd.DataFrame(N2gram.items(),columns=['lemma','frequency']).sort_values(by='frequency',ascending=False)

# load dataframe to csv at at output folder
dfresults = lemma_freq_N2

def load_N2grams_toCSV(df):
    # set relative path to folder
    cwd = os.getcwd()
    folder = '\\results\\'
    # set file
    csv_file = 'fin_lemma_N2grams_forSamples_csvtable_'
    #timestr = time.strftime("%Y%m%d-%H%M%S")
    csv_postfix = '.csv'
    #csv_source = cwd + folder + csv_file + timestr + csv_postfix
    csv_source = cwd + folder + csv_file + csv_postfix
    # get dataframe to CSV at defined source file ‘utf-8’
    df.to_csv(csv_source, index=False, encoding="utf-8-sig")

# Load dataframe to json at output folder
def load_N2grams_toJSON(df):
    # set relative path to folder
    cwd = os.getcwd()
    folder = '\\results\\'
    # set file
    json_file = 'fin_lemma_N2grams_forSamples_JSON_'
    #timestr = time.strftime("%Y%m%d-%H%M%S")
    json_postfix = '.json'
    #json_source = cwd + folder + json_file + timestr + json_postfix
    json_source = cwd + folder + json_file + json_postfix
    # get dataframe to json 
    result = df.to_json(orient='records')
    parsed = json.loads(result)
    parsed = {"fin_lemma_N2grams_forSamples": parsed}
    # Writing parsed data to json file
    json_object = json.dumps(parsed)
    with open(json_source, "w") as outfile:
        outfile.write(json_object)
        
# Load dataframe to excel and csv at defined source file
load_N2grams_toCSV(dfresults)
load_N2grams_toJSON(dfresults)

lemma_freq_N2.head(50)

,lemma,frequency
1285,vittu mikä,455
185,ei olla,222
141,se olla,150
1680,ei vittu,117
13252,paska ime,83
13253,ime paska,83
1108,ei se,81
136,joka olla,71
1408,olla se,68
298,että olla,66


## trigrams / 3-grams of 'raw' samples

In [25]:
# N3grams of tokenized, lemmatized and cleaned raw samples

all_n3grams = []
for sample in df['sampleN3grams']:
    for item in sample:
        all_n3grams.append(item)
all_n3grams

#count the lemma twograms
N3gram = Counter(all_n3grams)
N3gram

Counter({'ajatus se että': 1,
         'se että kaikki': 2,
         'että kaikki henkilö': 1,
         'kaikki henkilö tienata': 1,
         'henkilö tienata tulo': 1,
         'tienata tulo olla': 1,
         'tulo olla hän': 1,
         'olla hän oma': 1,
         'hän oma työ': 1,
         'oma työ tulos': 1,
         'työ tulos olla': 1,
         'tulos olla lähinnä': 1,
         'olla lähinnä naurettava': 1,
         'lähinnä naurettava se': 1,
         'naurettava se voida': 1,
         'se voida ehkä': 1,
         'voida ehkä päteä': 1,
         'ehkä päteä jos': 1,
         'päteä jos henkilö': 1,
         'jos henkilö elää': 1,
         'henkilö elää yksin': 1,
         'elää yksin metsä': 1,
         'yksin metsä susi': 1,
         'metsä susi kasvattaa': 1,
         'susi kasvattaa mutta': 1,
         'kasvattaa mutta ei': 1,
         'mutta ei moderni': 1,
         'ei moderni hyvinvointiyhteiskunta': 1,
         'moderni hyvinvointiyhteiskunta yhteiskunta': 1,
         'h

In [26]:
# create data frame for lemmas
lemma_freq_N3 = pd.DataFrame(N3gram.items(),columns=['lemma','frequency']).sort_values(by='frequency',ascending=False)

# load dataframe to csv at at output folder
dfresults = lemma_freq_N3

def load_N3grams_toCSV(df):
    # set relative path to folder
    cwd = os.getcwd()
    folder = '\\data\\'
    # set file
    csv_file = 'fin_lemma_N3grams_forSamples_csvtable_'
    #timestr = time.strftime("%Y%m%d-%H%M%S")
    csv_postfix = '.csv'
    #csv_source = cwd + folder + csv_file + timestr + csv_postfix
    csv_source = cwd + folder + csv_file + csv_postfix
    # get dataframe to CSV at defined source file ‘utf-8’
    df.to_csv(csv_source, index=False, encoding="utf-8-sig")

# Load dataframe to json at output folder
def load_N3grams_toJSON(df):
    # set relative path to folder
    cwd = os.getcwd()
    folder = '\\data\\'
    # set file
    json_file = 'fin_lemma_N3grams_forSamples_JSON_'
    #timestr = time.strftime("%Y%m%d-%H%M%S")
    json_postfix = '.json'
    #json_source = cwd + folder + json_file + timestr + json_postfix
    json_source = cwd + folder + json_file + json_postfix
    # get dataframe to json 
    result = df.to_json(orient='records')
    parsed = json.loads(result)
    parsed = {"fin_lemma_N3grams_forSamples": parsed}
    # Writing parsed data to json file
    json_object = json.dumps(parsed)
    with open(json_source, "w") as outfile:
        outfile.write(json_object)
        
# Load dataframe to excel and csv at defined source file
load_N3grams_toCSV(dfresults)
load_N3grams_toJSON(dfresults)

lemma_freq_N3.head(50)

,lemma,frequency
14029,paska ime paska,83
14030,ime paska ime,81
5352,ei vittu mikä,31
403,paikka joka olla,19
405,olla noobit homo,19
400,mopar olla paska,19
401,olla paska paikka,19
402,paska paikka joka,19
404,joka olla noobit,19
407,homo mopar olla,18


## fourgrams / 4-grams of 'raw' samples

In [27]:
# N4grams of tokenized, lemmatized and cleaned raw samples

all_n4grams = []
for sample in df['sampleN4grams']:
    for item in sample:
        all_n4grams.append(item)
all_n4grams

#count the lemma twograms
N4gram = Counter(all_n4grams)
N4gram

Counter({'ajatus se että kaikki': 1,
         'se että kaikki henkilö': 1,
         'että kaikki henkilö tienata': 1,
         'kaikki henkilö tienata tulo': 1,
         'henkilö tienata tulo olla': 1,
         'tienata tulo olla hän': 1,
         'tulo olla hän oma': 1,
         'olla hän oma työ': 1,
         'hän oma työ tulos': 1,
         'oma työ tulos olla': 1,
         'työ tulos olla lähinnä': 1,
         'tulos olla lähinnä naurettava': 1,
         'olla lähinnä naurettava se': 1,
         'lähinnä naurettava se voida': 1,
         'naurettava se voida ehkä': 1,
         'se voida ehkä päteä': 1,
         'voida ehkä päteä jos': 1,
         'ehkä päteä jos henkilö': 1,
         'päteä jos henkilö elää': 1,
         'jos henkilö elää yksin': 1,
         'henkilö elää yksin metsä': 1,
         'elää yksin metsä susi': 1,
         'yksin metsä susi kasvattaa': 1,
         'metsä susi kasvattaa mutta': 1,
         'susi kasvattaa mutta ei': 1,
         'kasvattaa mutta ei moderni

In [28]:
# create data frame for lemmas
lemma_freq_N4 = pd.DataFrame(N4gram.items(),columns=['lemma','frequency']).sort_values(by='frequency',ascending=False)

# load dataframe to csv at at output folder
dfresults = lemma_freq_N4

def load_N4grams_toCSV(df):
    # set relative path to folder
    cwd = os.getcwd()
    folder = '\\data\\'
    # set file
    csv_file = 'fin_lemma_N4grams_forSamples_csvtable_'
    #timestr = time.strftime("%Y%m%d-%H%M%S")
    csv_postfix = '.csv'
    #csv_source = cwd + folder + csv_file + timestr + csv_postfix
    csv_source = cwd + folder + csv_file + csv_postfix
    # get dataframe to CSV at defined source file ‘utf-8’
    df.to_csv(csv_source, index=False, encoding="utf-8-sig")

# Load dataframe to json at output folder
def load_N4grams_toJSON(df):
    # set relative path to folder
    cwd = os.getcwd()
    folder = '\\data\\'
    # set file
    json_file = 'fin_lemma_N4grams_forSamples_JSON_'
    #timestr = time.strftime("%Y%m%d-%H%M%S")
    json_postfix = '.json'
    #json_source = cwd + folder + json_file + timestr + json_postfix
    json_source = cwd + folder + json_file + json_postfix
    # get dataframe to json 
    result = df.to_json(orient='records')
    parsed = json.loads(result)
    parsed = {"fin_lemma_N4grams_forSamples": parsed}
    # Writing parsed data to json file
    json_object = json.dumps(parsed)
    with open(json_source, "w") as outfile:
        outfile.write(json_object)
        
# Load dataframe to excel and csv at defined source file
load_N4grams_toCSV(dfresults)
load_N4grams_toJSON(dfresults)

lemma_freq_N4.head(50)

,lemma,frequency
12880,paska ime paska ime,81
12881,ime paska ime paska,81
389,mopar olla paska paikka,19
390,olla paska paikka joka,19
391,paska paikka joka olla,19
392,paikka joka olla noobit,19
393,joka olla noobit homo,19
394,olla noobit homo mopar,18
395,noobit homo mopar olla,18
396,homo mopar olla paska,18


# Nxgram analysis for 'HS' class and 'not HS' class 
# - of 'correct' text samples

In [29]:
# Nxgram analysis for HS class and not HS class

df_notHS = df[df['HSbinary'] <= 0]
df_notHS

df_HS = df[df['HSbinary'] >= 1]
df_HS

,id,sample,sentiment,polarity,HSbinary,HSstrength,HStarget,HStopic,HSform,emotion,...,sampleLemmas,correctLemmas,sampleTokens,correctTokens,sampleN2grams,sampleN3grams,sampleN4grams,correctN2grams,correctN3grams,correctN4grams
2,3,--´973´¤-.ttu,2,-5,1,2,NONE,OTHER,"SWEARING, GRANULATED",CONTEMPT DISRESPECT,...,--´973´¤-.ttu,973 vittu,ttu,vittu,[],[],[],[],[],[]
3,4,-=>Widdu joo<=-,2,-4,1,2,NONE,TROLLING,"JOKE SARCASM, SWEARING, GRANULATED",SARCASTIC,...,-=>Widdu joo<= -,-= > vittu joo < = -,widdu joo,vittu joo,[widdu joo],[],[],[vittu joo],[],[]
5,6,- ei viddu -,2,-5,1,2,NONE,"OPINION, TROLLING","SWEARING, GRANULATED",SURPRISE,...,- ei viddu -,ei vittu,ei viddu,ei vittu,[ei viddu],[],[],[ei vittu],[],[]
7,8,"- Hyvä että JOKU näkee uuden , tyhmän veroperu...",2,-4,1,1,NONE,OPINION,SWEARING,"MIXED EMOTIONS, JOY, ANTICIPATION CRITICAL",...,"- hyvä että joku nähdä uusi , tyhmä veroperust...","hyvä että joku nähdä uusi , tyhmä veroperuste ...",hyvä että joku nähdä uusi tyhmä veroperuste jo...,hyvä että joku nähdä uusi tyhmä veroperuste jo...,"[hyvä että, että joku, joku nähdä, nähdä uusi,...","[hyvä että joku, että joku nähdä, joku nähdä u...","[hyvä että joku nähdä, että joku nähdä uusi, j...","[hyvä että, että joku, joku nähdä, nähdä uusi,...","[hyvä että joku, että joku nähdä, joku nähdä u...","[hyvä että joku nähdä, että joku nähdä uusi, j..."
11,12,- Tarpeeksi vihainen mies tappaa nyrkillä kius...,2,-5,1,3,GROUP,FAMILY SCHOOL FRIENDS,"INCITEMENT, VIOLENCE","ANGER HATE, AGGRESSIVE VIOLENT",...,- tarpeeksi vihainen mies tappaa nyrkki kiusaa...,tarpeeksi vihainen mies tappaa nyrkki kiusaaja...,tarpeeksi vihainen mies tappaa nyrkki kiusaaja...,tarpeeksi vihainen mies tappaa nyrkki kiusaaja...,"[tarpeeksi vihainen, vihainen mies, mies tappa...","[tarpeeksi vihainen mies, vihainen mies tappaa...","[tarpeeksi vihainen mies tappaa, vihainen mies...","[tarpeeksi vihainen, vihainen mies, mies tappa...","[tarpeeksi vihainen mies, vihainen mies tappaa...","[tarpeeksi vihainen mies tappaa, vihainen mies..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6611,6612,ööliä vittu,2,-4,1,2,NONE,"OPINION, TROLLING","SWEARING, GRANULATED",CONTEMPT DISRESPECT,...,ööliä vittu,olut vittu,ööliä vittu,olut vittu,[ööliä vittu],[],[],[olut vittu],[],[]
6612,6613,öö-luokkalaiset.vittuun,2,-5,1,2,NONE,TROLLING,"JOKE SARCASM, SWEARING",SARCASTIC,...,öö-luokkalaiset.vittuun,öö-luokkalaiset vittu,vittuun,vittu,[],[],[],[],[],[]
6613,6614,öö-vittu,2,-5,1,2,NONE,OTHER,"SWEARING, GRANULATED",DISGUST,...,öö-vittu,öö vittu,,öö vittu,[],[],[],[öö vittu],[],[]
6615,6616,Ööö mitä vittua??,2,-4,1,2,NONE,OPINION,"JOKE SARCASM, SWEARING",SURPRISE,...,Ööö mitä vittu ? ?,"Öö , mitä vittu ? ?",ööö mitä vittu,öö mitä vittu,"[ööö mitä, mitä vittu]",[ööö mitä vittu],[],"[öö mitä, mitä vittu]",[öö mitä vittu],[]


In [30]:
# ------------------------------------------------------#
# collect all not HS samples to one list
# N2grams of tokenized, lemmatized and cleaned raw samples

all_n2grams = []
for sample in df_HS['correctN2grams']:
    for word in sample:
        all_n2grams.append(word)
all_n2grams

#count the lemma twograms
N2gram = Counter(all_n2grams)
N2gram

Counter({'vittu joo': 52,
         'ei vittu': 208,
         'hyvä että': 4,
         'että joku': 6,
         'joku nähdä': 1,
         'nähdä uusi': 1,
         'uusi tyhmä': 1,
         'tyhmä veroperuste': 1,
         'veroperuste joka': 1,
         'joka tuijottaa': 1,
         'tuijottaa vain': 1,
         'vain co': 1,
         'co arvo': 1,
         'arvo ja': 1,
         'ja diesel': 1,
         'diesel paska': 1,
         'paska noki': 1,
         'noki ilma': 1,
         'ilma ja': 1,
         'ja keuhko': 1,
         'tarpeeksi vihainen': 1,
         'vihainen mies': 1,
         'mies tappaa': 1,
         'tappaa nyrkki': 1,
         'nyrkki kiusaaja': 1,
         'kiusaaja kun': 1,
         'kun mitta': 1,
         'mitta tulla': 1,
         'tulla täyteen': 1,
         'vittutieteen tohtori': 1,
         'käristyä helvetin': 1,
         'helvetin tuli': 1,
         'jos mies': 1,
         'mies vastaavasti': 1,
         'vastaavasti pukea': 1,
         'pukea itse': 1,
  

In [31]:
# create data frame for lemmas
lemma_freq_N2 = pd.DataFrame(N2gram.items(),columns=['lemma','frequency']).sort_values(by='frequency',ascending=False)

# load dataframe to csv at at output folder
dfresults = lemma_freq_N2

def load_N2grams_toCSV(df):
    # set relative path to folder
    cwd = os.getcwd()
    folder = '\\results\\'
    # set file
    csv_file = 'fin_lemma_N2grams_forHSonly_csvtable_'
    #timestr = time.strftime("%Y%m%d-%H%M%S")
    csv_postfix = '.csv'
    #csv_source = cwd + folder + csv_file + timestr + csv_postfix
    csv_source = cwd + folder + csv_file + csv_postfix
    # get dataframe to CSV at defined source file ‘utf-8’
    df.to_csv(csv_source, index=False, encoding="utf-8-sig")

# Load dataframe to json at output folder
def load_N2grams_toJSON(df):
    # set relative path to folder
    cwd = os.getcwd()
    folder = '\\results\\'
    # set file
    json_file = 'fin_lemma_N2grams_forHSonly_JSON_'
    #timestr = time.strftime("%Y%m%d-%H%M%S")
    json_postfix = '.json'
    #json_source = cwd + folder + json_file + timestr + json_postfix
    json_source = cwd + folder + json_file + json_postfix
    # get dataframe to json 
    result = df.to_json(orient='records')
    parsed = json.loads(result)
    parsed = {"fin_lemma_N2grams_forHSonly": parsed}
    # Writing parsed data to json file
    json_object = json.dumps(parsed)
    with open(json_source, "w") as outfile:
        outfile.write(json_object)
        
# Load dataframe to excel and csv at defined source file
load_N2grams_toCSV(dfresults)
load_N2grams_toJSON(dfresults)

lemma_freq_N2.head(50)

,lemma,frequency
315,vittu mikä,640
1,ei vittu,208
1338,ei olla,100
5387,paska ime,83
5388,ime paska,83
408,vittu kun,76
410,vittu että,68
229,se olla,66
455,vittu olla,58
76,että olla,57


In [32]:
# ------------------------------------------------------#
# collect all not HS samples to one list
# N3grams of tokenized, lemmatized and cleaned raw samples

all_n3grams = []
for sample in df_HS['correctN3grams']:
    for word in sample:
        all_n3grams.append(word)
all_n3grams

#count the lemma twograms
N3gram = Counter(all_n3grams)
N3gram

Counter({'hyvä että joku': 1,
         'että joku nähdä': 1,
         'joku nähdä uusi': 1,
         'nähdä uusi tyhmä': 1,
         'uusi tyhmä veroperuste': 1,
         'tyhmä veroperuste joka': 1,
         'veroperuste joka tuijottaa': 1,
         'joka tuijottaa vain': 1,
         'tuijottaa vain co': 1,
         'vain co arvo': 1,
         'co arvo ja': 1,
         'arvo ja diesel': 1,
         'ja diesel paska': 1,
         'diesel paska noki': 1,
         'paska noki ilma': 1,
         'noki ilma ja': 1,
         'ilma ja keuhko': 1,
         'tarpeeksi vihainen mies': 1,
         'vihainen mies tappaa': 1,
         'mies tappaa nyrkki': 1,
         'tappaa nyrkki kiusaaja': 1,
         'nyrkki kiusaaja kun': 1,
         'kiusaaja kun mitta': 1,
         'kun mitta tulla': 1,
         'mitta tulla täyteen': 1,
         'käristyä helvetin tuli': 1,
         'jos mies vastaavasti': 1,
         'mies vastaavasti pukea': 1,
         'vastaavasti pukea itse': 1,
         'pukea itse 

In [33]:
# create data frame for lemmas
lemma_freq_N3 = pd.DataFrame(N3gram.items(),columns=['lemma','frequency']).sort_values(by='frequency',ascending=False)

# load dataframe to csv at at output folder
dfresults = lemma_freq_N3

def load_N3grams_toCSV(df):
    # set relative path to folder
    cwd = os.getcwd()
    folder = '\\results\\'
    # set file
    csv_file = 'fin_lemma_N3grams_forHSonly_csvtable_'
    #timestr = time.strftime("%Y%m%d-%H%M%S")
    csv_postfix = '.csv'
    #csv_source = cwd + folder + csv_file + timestr + csv_postfix
    csv_source = cwd + folder + csv_file + csv_postfix
    # get dataframe to CSV at defined source file ‘utf-8’
    df.to_csv(csv_source, index=False, encoding="utf-8-sig")

# Load dataframe to json at output folder
def load_N3grams_toJSON(df):
    # set relative path to folder
    cwd = os.getcwd()
    folder = '\\results\\'
    # set file
    json_file = 'fin_lemma_N3grams_forHSonly_JSON_'
    #timestr = time.strftime("%Y%m%d-%H%M%S")
    json_postfix = '.json'
    #json_source = cwd + folder + json_file + timestr + json_postfix
    json_source = cwd + folder + json_file + json_postfix
    # get dataframe to json 
    result = df.to_json(orient='records')
    parsed = json.loads(result)
    parsed = {"fin_lemma_N3grams_forHSonly": parsed}
    # Writing parsed data to json file
    json_object = json.dumps(parsed)
    with open(json_source, "w") as outfile:
        outfile.write(json_object)
        
# Load dataframe to excel and csv at defined source file
load_N3grams_toCSV(dfresults)
load_N3grams_toJSON(dfresults)

lemma_freq_N3.head(50)

,lemma,frequency
5264,paska ime paska,83
5265,ime paska ime,81
1661,ei vittu mikä,45
338,vittu mikä paska,41
1027,vittu kun olla,32
464,vittu mikä pelle,31
458,vittu että olla,29
467,vittu mikä urpo,23
42,olla paska paikka,19
43,paska paikka joka,19


In [34]:
# ------------------------------------------------------#
# collect all not HS samples to one list
# N4grams of tokenized, lemmatized and cleaned raw samples

all_n4grams = []
for sample in df_HS['correctN4grams']:
    for word in sample:
        all_n4grams.append(word)
all_n4grams

#count the lemma twograms
N4gram = Counter(all_n4grams)
N4gram

Counter({'hyvä että joku nähdä': 1,
         'että joku nähdä uusi': 1,
         'joku nähdä uusi tyhmä': 1,
         'nähdä uusi tyhmä veroperuste': 1,
         'uusi tyhmä veroperuste joka': 1,
         'tyhmä veroperuste joka tuijottaa': 1,
         'veroperuste joka tuijottaa vain': 1,
         'joka tuijottaa vain co': 1,
         'tuijottaa vain co arvo': 1,
         'vain co arvo ja': 1,
         'co arvo ja diesel': 1,
         'arvo ja diesel paska': 1,
         'ja diesel paska noki': 1,
         'diesel paska noki ilma': 1,
         'paska noki ilma ja': 1,
         'noki ilma ja keuhko': 1,
         'tarpeeksi vihainen mies tappaa': 1,
         'vihainen mies tappaa nyrkki': 1,
         'mies tappaa nyrkki kiusaaja': 1,
         'tappaa nyrkki kiusaaja kun': 1,
         'nyrkki kiusaaja kun mitta': 1,
         'kiusaaja kun mitta tulla': 1,
         'kun mitta tulla täyteen': 1,
         'jos mies vastaavasti pukea': 1,
         'mies vastaavasti pukea itse': 1,
         'v

In [35]:
# create data frame for lemmas
lemma_freq_N4 = pd.DataFrame(N4gram.items(),columns=['lemma','frequency']).sort_values(by='frequency',ascending=False)

# load dataframe to csv at at output folder
dfresults = lemma_freq_N4

def load_N4grams_toCSV(df):
    # set relative path to folder
    cwd = os.getcwd()
    folder = '\\results\\'
    # set file
    csv_file = 'fin_lemma_N4grams_forHSonly_csvtable_'
    #timestr = time.strftime("%Y%m%d-%H%M%S")
    csv_postfix = '.csv'
    #csv_source = cwd + folder + csv_file + timestr + csv_postfix
    csv_source = cwd + folder + csv_file + csv_postfix
    # get dataframe to CSV at defined source file ‘utf-8’
    df.to_csv(csv_source, index=False, encoding="utf-8-sig")

# Load dataframe to json at output folder
def load_N4grams_toJSON(df):
    # set relative path to folder
    cwd = os.getcwd()
    folder = '\\results\\'
    # set file
    json_file = 'fin_lemma_N4grams_forHSonly_JSON_'
    #timestr = time.strftime("%Y%m%d-%H%M%S")
    json_postfix = '.json'
    #json_source = cwd + folder + json_file + timestr + json_postfix
    json_source = cwd + folder + json_file + json_postfix
    # get dataframe to json 
    result = df.to_json(orient='records')
    parsed = json.loads(result)
    parsed = {"fin_lemma_N4grams_forHSonly": parsed}
    # Writing parsed data to json file
    json_object = json.dumps(parsed)
    with open(json_source, "w") as outfile:
        outfile.write(json_object)
        
# Load dataframe to excel and csv at defined source file
load_N4grams_toCSV(dfresults)
load_N4grams_toJSON(dfresults)

lemma_freq_N4.head(50)

,lemma,frequency
4552,ime paska ime paska,81
4551,paska ime paska ime,81
41,joka olla noobit homo,19
40,paikka joka olla noobit,19
39,paska paikka joka olla,19
38,olla paska paikka joka,19
37,mopar olla paska paikka,19
43,noobit homo mopar olla,18
42,olla noobit homo mopar,18
44,homo mopar olla paska,18


## ----------------------------------------------
## ngram analysis of 'not HS' class for 'correct' samples

In [36]:
# ------------------------------------------------------#
# collect all not HS samples to one list
# N2grams of tokenized, lemmatized and cleaned raw samples

all_n2grams = []
for sample in df_notHS['correctN2grams']:
    for word in sample:
        all_n2grams.append(word)
all_n2grams

#count the lemma twograms
N2gram = Counter(all_n2grams)
N2gram

Counter({'ajatus se': 1,
         'se että': 47,
         'että kaikki': 5,
         'kaikki henkilö': 1,
         'henkilö tienata': 1,
         'tienata tulo': 1,
         'tulo olla': 1,
         'olla hän': 8,
         'hän oma': 2,
         'oma työ': 1,
         'työ tulos': 1,
         'tulos olla': 2,
         'olla lähinnä': 1,
         'lähinnä naurettava': 1,
         'naurettava se': 1,
         'se voida': 8,
         'voida ehkä': 1,
         'ehkä päteä': 1,
         'päteä jos': 1,
         'jos henkilö': 1,
         'henkilö elää': 2,
         'elää yksin': 2,
         'yksin metsä': 1,
         'metsä susi': 1,
         'susi kasvattaa': 1,
         'kasvattaa mutta': 1,
         'mutta ei': 22,
         'ei moderni': 1,
         'moderni hyvinvointiyhteiskunta': 1,
         'hyvinvointiyhteiskunta yhteiskunta': 1,
         'yhteiskunta olla': 1,
         'olla maksaa': 1,
         'maksaa lepomäki': 1,
         'lepomäki terveydenhoito': 1,
         'terveydenhoito j

In [37]:
# create data frame for lemmas
lemma_freq_N2 = pd.DataFrame(N2gram.items(),columns=['lemma','frequency']).sort_values(by='frequency',ascending=False)

# load dataframe to csv at at output folder
dfresults = lemma_freq_N2

def load_N2grams_toCSV(df):
    # set relative path to folder
    cwd = os.getcwd()
    folder = '\\results\\'
    # set file
    csv_file = 'fin_lemma_N2grams_forNotHS_csvtable_'
    #timestr = time.strftime("%Y%m%d-%H%M%S")
    csv_postfix = '.csv'
    #csv_source = cwd + folder + csv_file + timestr + csv_postfix
    csv_source = cwd + folder + csv_file + csv_postfix
    # get dataframe to CSV at defined source file ‘utf-8’
    df.to_csv(csv_source, index=False, encoding="utf-8-sig")

# Load dataframe to json at output folder
def load_N2grams_toJSON(df):
    # set relative path to folder
    cwd = os.getcwd()
    folder = '\\results\\'
    # set file
    json_file = 'fin_lemma_N2grams_forNotHS_JSON_'
    #timestr = time.strftime("%Y%m%d-%H%M%S")
    json_postfix = '.json'
    #json_source = cwd + folder + json_file + timestr + json_postfix
    json_source = cwd + folder + json_file + json_postfix
    # get dataframe to json 
    result = df.to_json(orient='records')
    parsed = json.loads(result)
    parsed = {"fin_lemma_N2grams_forNotHS": parsed}
    # Writing parsed data to json file
    json_object = json.dumps(parsed)
    with open(json_source, "w") as outfile:
        outfile.write(json_object)
        
# Load dataframe to excel and csv at defined source file
load_N2grams_toCSV(dfresults)
load_N2grams_toJSON(dfresults)

lemma_freq_N2.head(50)

,lemma,frequency
185,ei olla,159
141,se olla,95
1706,ei se,48
991,olla se,47
1,se että,47
1754,minä olla,43
154,ja se,37
136,joka olla,37
1041,olla ihana,37
363,ett ei,35


In [38]:
# ------------------------------------------------------#
# collect all not HS samples to one list
# N3grams of tokenized, lemmatized and cleaned raw samples

all_n3grams = []
for sample in df_notHS['correctN3grams']:
    for word in sample:
        all_n3grams.append(word)
all_n3grams

#count the lemma twograms
N3gram = Counter(all_n3grams)
N3gram

Counter({'ajatus se että': 1,
         'se että kaikki': 2,
         'että kaikki henkilö': 1,
         'kaikki henkilö tienata': 1,
         'henkilö tienata tulo': 1,
         'tienata tulo olla': 1,
         'tulo olla hän': 1,
         'olla hän oma': 1,
         'hän oma työ': 1,
         'oma työ tulos': 1,
         'työ tulos olla': 1,
         'tulos olla lähinnä': 1,
         'olla lähinnä naurettava': 1,
         'lähinnä naurettava se': 1,
         'naurettava se voida': 1,
         'se voida ehkä': 1,
         'voida ehkä päteä': 1,
         'ehkä päteä jos': 1,
         'päteä jos henkilö': 1,
         'jos henkilö elää': 1,
         'henkilö elää yksin': 1,
         'elää yksin metsä': 1,
         'yksin metsä susi': 1,
         'metsä susi kasvattaa': 1,
         'susi kasvattaa mutta': 1,
         'kasvattaa mutta ei': 1,
         'mutta ei moderni': 1,
         'ei moderni hyvinvointiyhteiskunta': 1,
         'moderni hyvinvointiyhteiskunta yhteiskunta': 1,
         'h

In [39]:
# create data frame for lemmas
lemma_freq_N3 = pd.DataFrame(N3gram.items(),columns=['lemma','frequency']).sort_values(by='frequency',ascending=False)

# load dataframe to csv at at output folder
dfresults = lemma_freq_N3

def load_N3grams_toCSV(df):
    # set relative path to folder
    cwd = os.getcwd()
    folder = '\\results\\'
    # set file
    csv_file = 'fin_lemma_N3grams_forNotHS_csvtable_'
    #timestr = time.strftime("%Y%m%d-%H%M%S")
    csv_postfix = '.csv'
    #csv_source = cwd + folder + csv_file + timestr + csv_postfix
    csv_source = cwd + folder + csv_file + csv_postfix
    # get dataframe to CSV at defined source file ‘utf-8’
    df.to_csv(csv_source, index=False, encoding="utf-8-sig")

# Load dataframe to json at output folder
def load_N3grams_toJSON(df):
    # set relative path to folder
    cwd = os.getcwd()
    folder = '\\results\\'
    # set file
    json_file = 'fin_lemma_N3grams_forNotHS_JSON_'
    #timestr = time.strftime("%Y%m%d-%H%M%S")
    json_postfix = '.json'
    #json_source = cwd + folder + json_file + timestr + json_postfix
    json_source = cwd + folder + json_file + json_postfix
    # get dataframe to json 
    result = df.to_json(orient='records')
    parsed = json.loads(result)
    parsed = {"fin_lemma_N3grams_forNotHS": parsed}
    # Writing parsed data to json file
    json_object = json.dumps(parsed)
    with open(json_source, "w") as outfile:
        outfile.write(json_object)
        
# Load dataframe to excel and csv at defined source file
load_N3grams_toCSV(dfresults)
load_N3grams_toJSON(dfresults)

lemma_freq_N3.head(50)

,lemma,frequency
932,ei olla mikään,9
508,olla aivan ihana,9
1066,olla se että,8
3000,se ei olla,7
2279,ei minä olla,7
1596,että se olla,6
3478,ei se olla,6
4377,voida sanoa että,6
4897,kuin vaan voi,6
10944,että minä olla,5


In [40]:
# ------------------------------------------------------#
# collect all not HS samples to one list
# N4grams of tokenized, lemmatized and cleaned raw samples

all_n4grams = []
for sample in df_notHS['correctN4grams']:
    for word in sample:
        all_n4grams.append(word)
all_n4grams

#count the lemma twograms
N4gram = Counter(all_n4grams)
N4gram

Counter({'ajatus se että kaikki': 1,
         'se että kaikki henkilö': 1,
         'että kaikki henkilö tienata': 1,
         'kaikki henkilö tienata tulo': 1,
         'henkilö tienata tulo olla': 1,
         'tienata tulo olla hän': 1,
         'tulo olla hän oma': 1,
         'olla hän oma työ': 1,
         'hän oma työ tulos': 1,
         'oma työ tulos olla': 1,
         'työ tulos olla lähinnä': 1,
         'tulos olla lähinnä naurettava': 1,
         'olla lähinnä naurettava se': 1,
         'lähinnä naurettava se voida': 1,
         'naurettava se voida ehkä': 1,
         'se voida ehkä päteä': 1,
         'voida ehkä päteä jos': 1,
         'ehkä päteä jos henkilö': 1,
         'päteä jos henkilö elää': 1,
         'jos henkilö elää yksin': 1,
         'henkilö elää yksin metsä': 1,
         'elää yksin metsä susi': 1,
         'yksin metsä susi kasvattaa': 1,
         'metsä susi kasvattaa mutta': 1,
         'susi kasvattaa mutta ei': 1,
         'kasvattaa mutta ei moderni

In [41]:
# create data frame for lemmas
lemma_freq_N4 = pd.DataFrame(N4gram.items(),columns=['lemma','frequency']).sort_values(by='frequency',ascending=False)

# load dataframe to csv at at output folder
dfresults = lemma_freq_N4

def load_N4grams_toCSV(df):
    # set relative path to folder
    cwd = os.getcwd()
    folder = '\\results\\'
    # set file
    csv_file = 'fin_lemma_N4grams_forNotHS_csvtable_'
    #timestr = time.strftime("%Y%m%d-%H%M%S")
    csv_postfix = '.csv'
    #csv_source = cwd + folder + csv_file + timestr + csv_postfix
    csv_source = cwd + folder + csv_file + csv_postfix
    # get dataframe to CSV at defined source file ‘utf-8’
    df.to_csv(csv_source, index=False, encoding="utf-8-sig")

# Load dataframe to json at output folder
def load_N4grams_toJSON(df):
    # set relative path to folder
    cwd = os.getcwd()
    folder = '\\results\\'
    # set file
    json_file = 'fin_lemma_N4grams_forNotHS_JSON_'
    #timestr = time.strftime("%Y%m%d-%H%M%S")
    json_postfix = '.json'
    #json_source = cwd + folder + json_file + timestr + json_postfix
    json_source = cwd + folder + json_file + json_postfix
    # get dataframe to json 
    result = df.to_json(orient='records')
    parsed = json.loads(result)
    parsed = {"fin_lemma_N4grams_forNotHS": parsed}
    # Writing parsed data to json file
    json_object = json.dumps(parsed)
    with open(json_source, "w") as outfile:
        outfile.write(json_object)
        
# Load dataframe to excel and csv at defined source file
load_N4grams_toCSV(dfresults)
load_N4grams_toJSON(dfresults)

lemma_freq_N4.head(50)

,lemma,frequency
6101,onnellinen aurinkoinen ihana ja,3
6100,maailma onnellinen aurinkoinen ihana,3
3300,olla muu elämä kuin,3
8455,olla se mieli että,3
17726,muu kun kunnon asenne,2
4539,ihana kun ei tarvita,2
4538,olla ihana kun ei,2
15559,suudelma ja lämmin halaus,2
17727,kun kunnon asenne elää,2
9816,kun ottaa huomio että,2


# -----------------------------------------------------------------------------------
# nxgram analysis for positive and negative class
# correct samples lemma analysis

In [42]:
# Nxgram analysis for positive class and negative class

df_positive = df[df['sentiment'] == 1]
df_positive

df_negative = df[df['sentiment'] == 2]
df_negative

,id,sample,sentiment,polarity,HSbinary,HSstrength,HStarget,HStopic,HSform,emotion,...,sampleLemmas,correctLemmas,sampleTokens,correctTokens,sampleN2grams,sampleN3grams,sampleN4grams,correctN2grams,correctN3grams,correctN4grams
0,1,- Ajatus siitä että kaikki henkilön tienaamat...,2,-1,0,0,NaN,NaN,NaN,"UNPLEASENT, ANTICIPATION CRITICAL, SARCASTIC",...,- ajatus se että kaikki henkilö tienata tulo...,- ajatus se että kaikki henkilö tienata tulo...,ajatus se että kaikki henkilö tienata tulo oll...,ajatus se että kaikki henkilö tienata tulo oll...,"[ajatus se, se että, että kaikki, kaikki henki...","[ajatus se että, se että kaikki, että kaikki h...","[ajatus se että kaikki, se että kaikki henkilö...","[ajatus se, se että, että kaikki, kaikki henki...","[ajatus se että, se että kaikki, että kaikki h...","[ajatus se että kaikki, se että kaikki henkilö..."
1,2,"- Kaivovertauksessa, viime hallitus on kaivan...",2,-2,0,0,NaN,NaN,NaN,"NEUTRAL NONE, UNPLEASENT, ANTICIPATION CRITICAL",...,"- kaivovertaus , viime hallitus olla kaivaa ...","- kaivovertaus , viime hallitus olla kaivaa ...",kaivovertaus viime hallitus olla kaivaa se kai...,kaivovertaus viime hallitus olla kaivaa se kai...,"[kaivovertaus viime, viime hallitus, hallitus ...","[kaivovertaus viime hallitus, viime hallitus o...","[kaivovertaus viime hallitus olla, viime halli...","[kaivovertaus viime, viime hallitus, hallitus ...","[kaivovertaus viime hallitus, viime hallitus o...","[kaivovertaus viime hallitus olla, viime halli..."
2,3,--´973´¤-.ttu,2,-5,1,2,NONE,OTHER,"SWEARING, GRANULATED",CONTEMPT DISRESPECT,...,--´973´¤-.ttu,973 vittu,ttu,vittu,[],[],[],[],[],[]
3,4,-=>Widdu joo<=-,2,-4,1,2,NONE,TROLLING,"JOKE SARCASM, SWEARING, GRANULATED",SARCASTIC,...,-=>Widdu joo<= -,-= > vittu joo < = -,widdu joo,vittu joo,[widdu joo],[],[],[vittu joo],[],[]
4,5,"- Ei , mutta olen joutunut elämään katsomalla ...",2,-5,0,0,NaN,NaN,NaN,"SADNESS, FEAR",...,"- ei , mutta olla joutua elää katsoa kuinka ki...","ei , mutta olla joutua elää katsoa kuinka kius...",ei mutta olla joutua elää katsoa kuinka kiusat...,ei mutta olla joutua elää katsoa kuinka kiusat...,"[ei mutta, mutta olla, olla joutua, joutua elä...","[ei mutta olla, mutta olla joutua, olla joutua...","[ei mutta olla joutua, mutta olla joutua elää,...","[ei mutta, mutta olla, olla joutua, joutua elä...","[ei mutta olla, mutta olla joutua, olla joutua...","[ei mutta olla joutua, mutta olla joutua elää,..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6616,6617,Ööövittuentiiä,2,-5,1,2,NONE,"OPINION, TROLLING","SWEARING, GRANULATED",CONTEMPT DISRESPECT,...,ööövittuentiiä,Öö vittu ei tietää,ööövittuentiiä,öö vittu ei tietää,[],[],[],"[öö vittu, vittu ei, ei tietää]","[öö vittu ei, vittu ei tietää]",[öö vittu ei tietää]
6619,6620,👎😠,2,-3,0,0,NaN,NaN,NaN,"UNPLEASENT, ANGER HATE",...,👎 😠,👎 😠,,,[],[],[],[],[],[]
6622,6623,😂 ai kauhee. Ei oo hauska asia jos itelle tapa...,2,-4,0,0,NaN,NaN,NaN,SURPRISE,...,😂 ai kauhee . ei oo hauska asia jos itelle tap...,😂 ai kauhee . ei oo hauska asia jos itelle tap...,ai kauhee ei oo hauska asia jos itelle tapahtu...,ai kauhee ei oo hauska asia jos itelle tapahtu...,"[ai kauhee, kauhee ei, ei oo, oo hauska, hausk...","[ai kauhee ei, kauhee ei oo, ei oo hauska, oo ...","[ai kauhee ei oo, kauhee ei oo hauska, ei oo h...","[ai kauhee, kauhee ei, ei oo, oo hauska, hausk...","[ai kauhee ei, kauhee ei oo, ei oo hauska, oo ...","[ai kauhee ei oo, kauhee ei oo hauska, ei oo h..."
6625,6626,😠👉👩‍💼,2,-3,0,0,NaN,NaN,NaN,ANGER HATE,...,😠 👉 👩 ‍ 💼,😠 👉 👩 ‍ 💼,,,[],[],[],[],[],[]


### nxgram analysis for 'positive' sentiment class of 'correct' samples

In [43]:
# ------------------------------------------------------#
# collect all 'positive' labeled correct samples to one list
# Nxgram analysis of tokenized, lemmatized and cleaned corrected samples

all_n2grams = []
for sample in df_positive['correctN2grams']:
    for word in sample:
        all_n2grams.append(word)
all_n2grams

#count the lemma twograms
N2gram = Counter(all_n2grams)
N2gram

Counter({'miks ei': 1,
         'ei sinä': 2,
         'sinä ottaa': 1,
         'ottaa vastaan': 1,
         'vastaan timo': 1,
         'timo jutila': 1,
         'jutila ja': 1,
         'ja tappara': 1,
         'tappara hurmoksellinen': 1,
         'hurmoksellinen joukkuehenki': 1,
         'hei vaan': 1,
         'vaan sinä': 1,
         'sinä ja': 5,
         'ja onnittelu': 1,
         'ikärajoittaa video': 1,
         'näin minä': 1,
         'minä vihreä': 1,
         'vihreä ajatella': 1,
         'ajatella hauska': 1,
         'hauska pätkä': 1,
         'olla se': 11,
         'se mies': 1,
         'mies xd': 1,
         'sotanorsu ei': 1,
         'ei kestää': 1,
         'kestää ps': 1,
         'ps kukaan': 1,
         'kukaan ihminen': 1,
         'ihminen ei': 1,
         'ei voida': 3,
         'voida olla': 5,
         'olla ruma': 2,
         'ruma kaikki': 1,
         'kaikki olla': 5,
         'olla kaunis': 5,
         'kaunis mutta': 1,
         'mutta vaan': 

In [44]:
# create data frame for lemmas
lemma_freq_N2 = pd.DataFrame(N2gram.items(),columns=['lemma','frequency']).sort_values(by='frequency',ascending=False)

# load dataframe to csv at at output folder
dfresults = lemma_freq_N2

def load_N2grams_toCSV(df):
    # set relative path to folder
    cwd = os.getcwd()
    folder = '\\results\\'
    # set file
    csv_file = 'fin_lemma_N2grams_positiveClass_csvtable_'
    #timestr = time.strftime("%Y%m%d-%H%M%S")
    csv_postfix = '.csv'
    #csv_source = cwd + folder + csv_file + timestr + csv_postfix
    csv_source = cwd + folder + csv_file + csv_postfix
    # get dataframe to CSV at defined source file ‘utf-8’
    df.to_csv(csv_source, index=False, encoding="utf-8-sig")

# Load dataframe to json at output folder
def load_N2grams_toJSON(df):
    # set relative path to folder
    cwd = os.getcwd()
    folder = '\\results\\'
    # set file
    json_file = 'fin_lemma_N2grams_positiveClass_JSON_'
    #timestr = time.strftime("%Y%m%d-%H%M%S")
    json_postfix = '.json'
    #json_source = cwd + folder + json_file + timestr + json_postfix
    json_source = cwd + folder + json_file + json_postfix
    # get dataframe to json 
    result = df.to_json(orient='records')
    parsed = json.loads(result)
    parsed = {"fin_lemma_N2grams_positiveClass": parsed}
    # Writing parsed data to json file
    json_object = json.dumps(parsed)
    with open(json_source, "w") as outfile:
        outfile.write(json_object)
        
# Load dataframe to excel and csv at defined source file
load_N2grams_toCSV(dfresults)
load_N2grams_toJSON(dfresults)

lemma_freq_N2.head(50)

,lemma,frequency
42,olla ihana,28
49,ei olla,23
287,se olla,22
215,minä olla,16
910,ihana ja,15
142,aivan ihana,14
852,ja ihana,14
220,olla hyvä,14
755,ei vittu,12
60,hän olla,12


In [45]:
# ------------------------------------------------------#
# collect all 'positive' labeled correct samples to one list
# Nxgram analysis of tokenized, lemmatized and cleaned corrected samples

all_n3grams = []
for sample in df_positive['correctN3grams']:
    for word in sample:
        all_n3grams.append(word)
all_n3grams

#count the lemma twograms
N3gram = Counter(all_n3grams)
N3gram

Counter({'miks ei sinä': 1,
         'ei sinä ottaa': 1,
         'sinä ottaa vastaan': 1,
         'ottaa vastaan timo': 1,
         'vastaan timo jutila': 1,
         'timo jutila ja': 1,
         'jutila ja tappara': 1,
         'ja tappara hurmoksellinen': 1,
         'tappara hurmoksellinen joukkuehenki': 1,
         'hei vaan sinä': 1,
         'vaan sinä ja': 1,
         'sinä ja onnittelu': 1,
         'näin minä vihreä': 1,
         'minä vihreä ajatella': 1,
         'vihreä ajatella hauska': 1,
         'ajatella hauska pätkä': 1,
         'olla se mies': 1,
         'se mies xd': 1,
         'sotanorsu ei kestää': 1,
         'ei kestää ps': 1,
         'kestää ps kukaan': 1,
         'ps kukaan ihminen': 1,
         'kukaan ihminen ei': 1,
         'ihminen ei voida': 1,
         'ei voida olla': 2,
         'voida olla ruma': 1,
         'olla ruma kaikki': 1,
         'ruma kaikki olla': 1,
         'kaikki olla kaunis': 1,
         'olla kaunis mutta': 1,
         'kaun

In [46]:
# create data frame for lemmas
lemma_freq_N3 = pd.DataFrame(N3gram.items(),columns=['lemma','frequency']).sort_values(by='frequency',ascending=False)

# load dataframe to csv at at output folder
dfresults = lemma_freq_N3

def load_N3grams_toCSV(df):
    # set relative path to folder
    cwd = os.getcwd()
    folder = '\\results\\'
    # set file
    csv_file = 'fin_lemma_N3grams_positiveClass_csvtable_'
    #timestr = time.strftime("%Y%m%d-%H%M%S")
    csv_postfix = '.csv'
    #csv_source = cwd + folder + csv_file + timestr + csv_postfix
    csv_source = cwd + folder + csv_file + csv_postfix
    # get dataframe to CSV at defined source file ‘utf-8’
    df.to_csv(csv_source, index=False, encoding="utf-8-sig")

# Load dataframe to json at output folder
def load_N3grams_toJSON(df):
    # set relative path to folder
    cwd = os.getcwd()
    folder = '\\results\\'
    # set file
    json_file = 'fin_lemma_N3grams_positiveClass_JSON_'
    #timestr = time.strftime("%Y%m%d-%H%M%S")
    json_postfix = '.json'
    #json_source = cwd + folder + json_file + timestr + json_postfix
    json_source = cwd + folder + json_file + json_postfix
    # get dataframe to json 
    result = df.to_json(orient='records')
    parsed = json.loads(result)
    parsed = {"fin_lemma_positiveClass": parsed}
    # Writing parsed data to json file
    json_object = json.dumps(parsed)
    with open(json_source, "w") as outfile:
        outfile.write(json_object)
        
# Load dataframe to excel and csv at defined source file
load_N3grams_toCSV(dfresults)
load_N3grams_toJSON(dfresults)

lemma_freq_N3.head(50)

,lemma,frequency
687,ei vittu mage,11
125,olla aivan ihana,7
809,kuin vaan voi,6
660,elämä olla ihana,5
805,onnellinen aurinkoinen ihana,5
922,ei olla mikään,4
1859,ihana kuultaa että,3
1138,aurinkoinen ihana ja,3
1137,maailma onnellinen aurinkoinen,3
688,vittu mage ddd,3


In [47]:
# ------------------------------------------------------#
# collect all 'positive' labeled correct samples to one list
# Nxgram analysis of tokenized, lemmatized and cleaned corrected samples

all_n4grams = []
for sample in df_positive['correctN4grams']:
    for word in sample:
        all_n4grams.append(word)
all_n4grams

#count the lemma twograms
N4gram = Counter(all_n4grams)
N4gram

Counter({'miks ei sinä ottaa': 1,
         'ei sinä ottaa vastaan': 1,
         'sinä ottaa vastaan timo': 1,
         'ottaa vastaan timo jutila': 1,
         'vastaan timo jutila ja': 1,
         'timo jutila ja tappara': 1,
         'jutila ja tappara hurmoksellinen': 1,
         'ja tappara hurmoksellinen joukkuehenki': 1,
         'hei vaan sinä ja': 1,
         'vaan sinä ja onnittelu': 1,
         'näin minä vihreä ajatella': 1,
         'minä vihreä ajatella hauska': 1,
         'vihreä ajatella hauska pätkä': 1,
         'olla se mies xd': 1,
         'sotanorsu ei kestää ps': 1,
         'ei kestää ps kukaan': 1,
         'kestää ps kukaan ihminen': 1,
         'ps kukaan ihminen ei': 1,
         'kukaan ihminen ei voida': 1,
         'ihminen ei voida olla': 1,
         'ei voida olla ruma': 1,
         'voida olla ruma kaikki': 1,
         'olla ruma kaikki olla': 1,
         'ruma kaikki olla kaunis': 1,
         'kaikki olla kaunis mutta': 1,
         'olla kaunis mutta v

In [48]:
# create data frame for lemmas
lemma_freq_N4 = pd.DataFrame(N4gram.items(),columns=['lemma','frequency']).sort_values(by='frequency',ascending=False)

# load dataframe to csv at output folder
dfresults = lemma_freq_N4

def load_N4grams_toCSV(df):
    # set relative path to folder
    cwd = os.getcwd()
    folder = '\\results\\'
    # set file
    csv_file = 'fin_lemma_N4grams_positiveClass_csvtable_'
    #timestr = time.strftime("%Y%m%d-%H%M%S")
    csv_postfix = '.csv'
    #csv_source = cwd + folder + csv_file + timestr + csv_postfix
    csv_source = cwd + folder + csv_file + csv_postfix
    # get dataframe to CSV at defined source file ‘utf-8’
    df.to_csv(csv_source, index=False, encoding="utf-8-sig")

# Load dataframe to json at output folder
def load_N4grams_toJSON(df):
    # set relative path to folder
    cwd = os.getcwd()
    folder = '\\results\\'
    # set file
    json_file = 'fin_lemma_N4grams_positiveClass_JSON_'
    #timestr = time.strftime("%Y%m%d-%H%M%S")
    json_postfix = '.json'
    #json_source = cwd + folder + json_file + timestr + json_postfix
    json_source = cwd + folder + json_file + json_postfix
    # get dataframe to json 
    result = df.to_json(orient='records')
    parsed = json.loads(result)
    parsed = {"fin_lemma_positiveClass": parsed}
    # Writing parsed data to json file
    json_object = json.dumps(parsed)
    with open(json_source, "w") as outfile:
        outfile.write(json_object)
        
# Load dataframe to excel and csv at defined source file
load_N4grams_toCSV(dfresults)
load_N4grams_toJSON(dfresults)

print('Length of result table: ', len(dfresults))
lemma_freq_N4.head(50)

Length of result table:  4663


,lemma,frequency
1028,maailma onnellinen aurinkoinen ihana,3
1029,onnellinen aurinkoinen ihana ja,3
613,ei vittu mage ddd,3
1620,ja olla sanoa että,2
1607,sanoa että hän olla,2
965,ihana mikä yhdessä keksiä,2
3731,rakastaa kunnioittaa ja ihailla,2
4025,sellainen kun olla ja,2
768,ihana kun ei tarvita,2
767,olla ihana kun ei,2


### -----------------------------------------------------------------------------------
### nxgram analysis for 'negative' sentiment class of 'correct' samples

In [49]:
# ------------------------------------------------------#
# collect all 'negative' labeled correct samples to one list
# Nxgram analysis of tokenized, lemmatized and cleaned corrected samples

all_n2grams = []
for sample in df_negative['correctN2grams']:
    for word in sample:
        all_n2grams.append(word)
all_n2grams

#count the lemma twograms
N2gram = Counter(all_n2grams)
N2gram

Counter({'ajatus se': 1,
         'se että': 53,
         'että kaikki': 5,
         'kaikki henkilö': 1,
         'henkilö tienata': 1,
         'tienata tulo': 1,
         'tulo olla': 1,
         'olla hän': 7,
         'hän oma': 3,
         'oma työ': 1,
         'työ tulos': 1,
         'tulos olla': 1,
         'olla lähinnä': 1,
         'lähinnä naurettava': 1,
         'naurettava se': 1,
         'se voida': 8,
         'voida ehkä': 2,
         'ehkä päteä': 1,
         'päteä jos': 1,
         'jos henkilö': 1,
         'henkilö elää': 2,
         'elää yksin': 2,
         'yksin metsä': 1,
         'metsä susi': 1,
         'susi kasvattaa': 1,
         'kasvattaa mutta': 1,
         'mutta ei': 21,
         'ei moderni': 1,
         'moderni hyvinvointiyhteiskunta': 1,
         'hyvinvointiyhteiskunta yhteiskunta': 1,
         'yhteiskunta olla': 1,
         'olla maksaa': 2,
         'maksaa lepomäki': 1,
         'lepomäki terveydenhoito': 1,
         'terveydenhoito j

In [50]:
# create data frame for lemmas
lemma_freq_N2 = pd.DataFrame(N2gram.items(),columns=['lemma','frequency']).sort_values(by='frequency',ascending=False)

# load dataframe to csv at at output folder
dfresults = lemma_freq_N2

def load_N2grams_toCSV(df):
    # set relative path to folder
    cwd = os.getcwd()
    folder = '\\results\\'
    # set file
    csv_file = 'fin_lemma_N2grams_negativeClass_csvtable_'
    #timestr = time.strftime("%Y%m%d-%H%M%S")
    csv_postfix = '.csv'
    #csv_source = cwd + folder + csv_file + timestr + csv_postfix
    csv_source = cwd + folder + csv_file + csv_postfix
    # get dataframe to CSV at defined source file ‘utf-8’
    df.to_csv(csv_source, index=False, encoding="utf-8-sig")

# Load dataframe to json at output folder
def load_N2grams_toJSON(df):
    # set relative path to folder
    cwd = os.getcwd()
    folder = '\\results\\'
    # set file
    json_file = 'fin_lemma_N2grams_negativeClass_JSON_'
    #timestr = time.strftime("%Y%m%d-%H%M%S")
    json_postfix = '.json'
    #json_source = cwd + folder + json_file + timestr + json_postfix
    json_source = cwd + folder + json_file + json_postfix
    # get dataframe to json 
    result = df.to_json(orient='records')
    parsed = json.loads(result)
    parsed = {"fin_lemma_N2grams_negativeClass": parsed}
    # Writing parsed data to json file
    json_object = json.dumps(parsed)
    with open(json_source, "w") as outfile:
        outfile.write(json_object)
        
# Load dataframe to excel and csv at defined source file
load_N2grams_toCSV(dfresults)
load_N2grams_toJSON(dfresults)

lemma_freq_N2.head(50)

,lemma,frequency
999,vittu mikä,636
235,ei vittu,194
185,ei olla,181
141,se olla,101
9676,paska ime,83
9677,ime paska,83
1176,vittu kun,75
1184,vittu että,67
294,että olla,64
578,kun olla,63


In [51]:
# ------------------------------------------------------#
# collect all 'negative' labeled correct samples to one list
# Nxgram analysis of tokenized, lemmatized and cleaned corrected samples

all_n3grams = []
for sample in df_negative['correctN3grams']:
#for sample in df_negative['sampleN3grams']:
    for word in sample:
        all_n3grams.append(word)
all_n3grams

#count the lemma twograms
N3gram = Counter(all_n3grams)
N3gram

Counter({'ajatus se että': 1,
         'se että kaikki': 2,
         'että kaikki henkilö': 1,
         'kaikki henkilö tienata': 1,
         'henkilö tienata tulo': 1,
         'tienata tulo olla': 1,
         'tulo olla hän': 1,
         'olla hän oma': 1,
         'hän oma työ': 1,
         'oma työ tulos': 1,
         'työ tulos olla': 1,
         'tulos olla lähinnä': 1,
         'olla lähinnä naurettava': 1,
         'lähinnä naurettava se': 1,
         'naurettava se voida': 1,
         'se voida ehkä': 1,
         'voida ehkä päteä': 1,
         'ehkä päteä jos': 1,
         'päteä jos henkilö': 1,
         'jos henkilö elää': 1,
         'henkilö elää yksin': 1,
         'elää yksin metsä': 1,
         'yksin metsä susi': 1,
         'metsä susi kasvattaa': 1,
         'susi kasvattaa mutta': 1,
         'kasvattaa mutta ei': 1,
         'mutta ei moderni': 1,
         'ei moderni hyvinvointiyhteiskunta': 1,
         'moderni hyvinvointiyhteiskunta yhteiskunta': 1,
         'h

In [52]:
# create data frame for lemmas
lemma_freq_N3 = pd.DataFrame(N3gram.items(),columns=['lemma','frequency']).sort_values(by='frequency',ascending=False)

# load dataframe to csv at at output folder
dfresults = lemma_freq_N3

def load_N3grams_toCSV(df):
    # set relative path to folder
    cwd = os.getcwd()
    folder = '\\results\\'
    # set file
    csv_file = 'fin_lemma_N3grams_negativeClass_csvtable_'
    #timestr = time.strftime("%Y%m%d-%H%M%S")
    csv_postfix = '.csv'
    #csv_source = cwd + folder + csv_file + timestr + csv_postfix
    csv_source = cwd + folder + csv_file + csv_postfix
    # get dataframe to CSV at defined source file ‘utf-8’
    df.to_csv(csv_source, index=False, encoding="utf-8-sig")

# Load dataframe to json at output folder
def load_N3grams_toJSON(df):
    # set relative path to folder
    cwd = os.getcwd()
    folder = '\\results\\'
    # set file
    json_file = 'fin_lemma_N3grams_negativeClass_JSON_'
    #timestr = time.strftime("%Y%m%d-%H%M%S")
    json_postfix = '.json'
    #json_source = cwd + folder + json_file + timestr + json_postfix
    json_source = cwd + folder + json_file + json_postfix
    # get dataframe to json 
    result = df.to_json(orient='records')
    parsed = json.loads(result)
    parsed = {"fin_lemma_N3grams_negativeClass": parsed}
    # Writing parsed data to json file
    json_object = json.dumps(parsed)
    with open(json_source, "w") as outfile:
        outfile.write(json_object)
        
# Load dataframe to excel and csv at defined source file
load_N3grams_toCSV(dfresults)
load_N3grams_toJSON(dfresults)

lemma_freq_N3.head(50)

,lemma,frequency
10015,paska ime paska,83
10016,ime paska ime,81
3325,ei vittu mikä,45
1085,vittu mikä paska,41
2440,vittu kun olla,32
1389,vittu mikä pelle,31
1383,vittu että olla,28
1392,vittu mikä urpo,23
401,paikka joka olla,19
399,olla paska paikka,19


In [53]:
# ------------------------------------------------------#
# collect all 'positive' labeled correct samples to one list
# Nxgram analysis of tokenized, lemmatized and cleaned corrected samples

all_n4grams = []
for sample in df_negative['correctN4grams']:
    for word in sample:
        all_n4grams.append(word)
all_n4grams

#count the lemma twograms
N4gram = Counter(all_n4grams)
N4gram

Counter({'ajatus se että kaikki': 1,
         'se että kaikki henkilö': 1,
         'että kaikki henkilö tienata': 1,
         'kaikki henkilö tienata tulo': 1,
         'henkilö tienata tulo olla': 1,
         'tienata tulo olla hän': 1,
         'tulo olla hän oma': 1,
         'olla hän oma työ': 1,
         'hän oma työ tulos': 1,
         'oma työ tulos olla': 1,
         'työ tulos olla lähinnä': 1,
         'tulos olla lähinnä naurettava': 1,
         'olla lähinnä naurettava se': 1,
         'lähinnä naurettava se voida': 1,
         'naurettava se voida ehkä': 1,
         'se voida ehkä päteä': 1,
         'voida ehkä päteä jos': 1,
         'ehkä päteä jos henkilö': 1,
         'päteä jos henkilö elää': 1,
         'jos henkilö elää yksin': 1,
         'henkilö elää yksin metsä': 1,
         'elää yksin metsä susi': 1,
         'yksin metsä susi kasvattaa': 1,
         'metsä susi kasvattaa mutta': 1,
         'susi kasvattaa mutta ei': 1,
         'kasvattaa mutta ei moderni

In [54]:
# create data frame for lemmas
lemma_freq_N4 = pd.DataFrame(N4gram.items(),columns=['lemma','frequency']).sort_values(by='frequency',ascending=False)

# load dataframe to csv at at output folder
dfresults = lemma_freq_N4

def load_N4grams_toCSV(df):
    # set relative path to folder
    cwd = os.getcwd()
    folder = '\\results\\'
    # set file
    csv_file = 'fin_lemma_N4grams_negativeClass_csvtable_'
    #timestr = time.strftime("%Y%m%d-%H%M%S")
    csv_postfix = '.csv'
    #csv_source = cwd + folder + csv_file + timestr + csv_postfix
    csv_source = cwd + folder + csv_file + csv_postfix
    # get dataframe to CSV at defined source file ‘utf-8’
    df.to_csv(csv_source, index=False, encoding="utf-8-sig")

# Load dataframe to json at output folder
def load_N4grams_toJSON(df):
    # set relative path to folder
    cwd = os.getcwd()
    folder = '\\results\\'
    # set file
    json_file = 'fin_lemma_N4grams_negativeClass_JSON_'
    #timestr = time.strftime("%Y%m%d-%H%M%S")
    json_postfix = '.json'
    #json_source = cwd + folder + json_file + timestr + json_postfix
    json_source = cwd + folder + json_file + json_postfix
    # get dataframe to json 
    result = df.to_json(orient='records')
    parsed = json.loads(result)
    parsed = {"fin_lemma_N4grams_negativeClass": parsed}
    # Writing parsed data to json file
    json_object = json.dumps(parsed)
    with open(json_source, "w") as outfile:
        outfile.write(json_object)
        
# Load dataframe to excel and csv at defined source file
load_N4grams_toCSV(dfresults)
load_N4grams_toJSON(dfresults)

lemma_freq_N4.head(50)

,lemma,frequency
8997,paska ime paska ime,81
8998,ime paska ime paska,81
389,mopar olla paska paikka,19
390,olla paska paikka joka,19
391,paska paikka joka olla,19
392,paikka joka olla noobit,19
393,joka olla noobit homo,19
396,homo mopar olla paska,18
395,noobit homo mopar olla,18
394,olla noobit homo mopar,18


# -----------------------------------------------------------------------------------
# Nxgram analysis for 'raw' samples
## raw samples analysis - no cleaning, no lemmatization

In [55]:
# import data from xlsx file to pd dataFolder
cwd = os.getcwd()
folder = '\\data\\'
#csv_file = 'HSTrainData.xlsx'
csv_file = 'Manually_Annotated_Collection_ALL_FINAL_22023Merja.xlsx'
csv_source = cwd + folder + csv_file
#print(csv_source)
df = pd.read_excel(csv_source)
df

,id,sample,sentiment,polarity,HSbinary,HSstrength,HStarget,HStopic,HSform,emotion,urban_finnish,correct_finnish,user_nick
0,1,- Ajatus siitä että kaikki henkilön tienaamat...,negative,-1,False,0,NaN,NaN,NaN,"UNPLEASENT, ANTICIPATION CRITICAL, SARCASTIC",- Ajatus siitä että kaikki henkilön tienaamat...,- Ajatus siitä että kaikki henkilön tienaamat...,Meria
1,2,"- Kaivovertauksessa, viime hallitus on kaivan...",negative,-2,False,0,NaN,NaN,NaN,"NEUTRAL NONE, UNPLEASENT, ANTICIPATION CRITICAL","- Kaivovertauksessa, viime hallitus on kaivan...","- Kaivovertauksessa, viime hallitus on kaivan...",Meria
2,3,--´973´¤-.ttu,negative,-5,True,2,NONE,OTHER,"SWEARING, GRANULATED",CONTEMPT DISRESPECT,--´973´¤-.ttu,973 vittu,Meria
3,4,-=>Widdu joo<=-,negative,-4,True,2,NONE,TROLLING,"JOKE SARCASM, SWEARING, GRANULATED",SARCASTIC,-=> Widdu joo <=-,-=> vittu joo <=-,Meria
4,5,"- Ei , mutta olen joutunut elämään katsomalla ...",negative,-5,False,0,NaN,NaN,NaN,"SADNESS, FEAR","- Ei , mutta olen joutunut elämään katsomalla ...","Ei , mutta olen joutunut elämään katsomalla ku...",Meria
...,...,...,...,...,...,...,...,...,...,...,...,...,...
6624,6625,😂😂😂😂😂😂😂😂😂😂😂😂😂😂,positive,5,False,0,NaN,NaN,NaN,JOY,😂😂😂😂😂😂😂😂😂😂😂😂😂😂,😂😂😂😂😂,Meria
6625,6626,😠👉👩‍💼,negative,-3,False,0,NaN,NaN,NaN,ANGER HATE,😠👉👩‍💼,😠👉👩‍💼,Meria
6626,6627,🤣🤣🤣🤣🤣🤣🤣🤣🤣🤣,positive,5,False,0,NaN,NaN,NaN,JOY,🤣🤣🤣🤣🤣🤣🤣🤣🤣🤣,🤣🤣🤣🤣🤣🤣🤣🤣🤣🤣,Meria
6627,6628,🤮😡,negative,-4,False,0,NaN,NaN,NaN,"DISGUST, ANGER HATE",🤮😡,🤮😡,Meria


In [56]:
# Change text samples to string
df["stringSample"] = df[str("sample")]
df["stringCorrect"] = df[str("correct_finnish")]

# add string len column
df["stringLength"] = ""

# extract string length measures of data
def extract_stringLen(df):
    for item in df["stringCorrect"]:
        #newframe["stringLength"] = len(str(item))
        df["stringLength"] = df.stringCorrect.str.len()
    return df

extract_stringLen(df)
df.head(50)

,id,sample,sentiment,polarity,HSbinary,HSstrength,HStarget,HStopic,HSform,emotion,urban_finnish,correct_finnish,user_nick,stringSample,stringCorrect,stringLength
0,1,- Ajatus siitä että kaikki henkilön tienaamat...,negative,-1,False,0,NaN,NaN,NaN,"UNPLEASENT, ANTICIPATION CRITICAL, SARCASTIC",- Ajatus siitä että kaikki henkilön tienaamat...,- Ajatus siitä että kaikki henkilön tienaamat...,Meria,- Ajatus siitä että kaikki henkilön tienaamat...,- Ajatus siitä että kaikki henkilön tienaamat...,866
1,2,"- Kaivovertauksessa, viime hallitus on kaivan...",negative,-2,False,0,NaN,NaN,NaN,"NEUTRAL NONE, UNPLEASENT, ANTICIPATION CRITICAL","- Kaivovertauksessa, viime hallitus on kaivan...","- Kaivovertauksessa, viime hallitus on kaivan...",Meria,"- Kaivovertauksessa, viime hallitus on kaivan...","- Kaivovertauksessa, viime hallitus on kaivan...",984
2,3,--´973´¤-.ttu,negative,-5,True,2,NONE,OTHER,"SWEARING, GRANULATED",CONTEMPT DISRESPECT,--´973´¤-.ttu,973 vittu,Meria,--´973´¤-.ttu,973 vittu,9
3,4,-=>Widdu joo<=-,negative,-4,True,2,NONE,TROLLING,"JOKE SARCASM, SWEARING, GRANULATED",SARCASTIC,-=> Widdu joo <=-,-=> vittu joo <=-,Meria,-=>Widdu joo<=-,-=> vittu joo <=-,17
4,5,"- Ei , mutta olen joutunut elämään katsomalla ...",negative,-5,False,0,NaN,NaN,NaN,"SADNESS, FEAR","- Ei , mutta olen joutunut elämään katsomalla ...","Ei , mutta olen joutunut elämään katsomalla ku...",Meria,"- Ei , mutta olen joutunut elämään katsomalla ...","Ei , mutta olen joutunut elämään katsomalla ku...",144
5,6,- ei viddu -,negative,-5,True,2,NONE,"OPINION, TROLLING","SWEARING, GRANULATED",SURPRISE,ei viddu,ei vittu,Meria,- ei viddu -,ei vittu,8
6,7,- Huvittunut -,positive,1,False,0,NaN,NaN,NaN,JOY,Huvittunut,huvittunut,Meria,- Huvittunut -,huvittunut,10
7,8,"- Hyvä että JOKU näkee uuden , tyhmän veroperu...",negative,-4,True,1,NONE,OPINION,SWEARING,"MIXED EMOTIONS, JOY, ANTICIPATION CRITICAL","- Hyvä että JOKU näkee uuden , tyhmän veroperu...","Hyvä että JOKU näkee uuden, tyhmän veroperuste...",Meria,"- Hyvä että JOKU näkee uuden , tyhmän veroperu...","Hyvä että JOKU näkee uuden, tyhmän veroperuste...",124
8,9,- Kukaan luterilainen ei korota itseään hänen ...,negative,-2,False,0,NaN,NaN,NaN,SADNESS,- Kukaan luterilainen ei korota itseään hänen ...,Kukaan luterilainen ei korota itseään hänen su...,Meria,- Kukaan luterilainen ei korota itseään hänen ...,Kukaan luterilainen ei korota itseään hänen su...,105
9,10,- kuten on sovittu -,neutral,0,False,0,NaN,NaN,NaN,NEUTRAL NONE,- kuten on sovittu -,kuten on sovittu,Meria,- kuten on sovittu -,kuten on sovittu,16


In [57]:
# change sentiment values
df['sentiment'] = df['sentiment'].replace('neutral', 0)
df['sentiment'] = df['sentiment'].replace('positive', 1)
df['sentiment'] = df['sentiment'].replace('negative', 2)

# change HS / not HS values
df['HSbinary'] = df['HSbinary'].replace(False, 0)
df['HSbinary'] = df['HSbinary'].replace(True, 1)
df

,id,sample,sentiment,polarity,HSbinary,HSstrength,HStarget,HStopic,HSform,emotion,urban_finnish,correct_finnish,user_nick,stringSample,stringCorrect,stringLength
0,1,- Ajatus siitä että kaikki henkilön tienaamat...,2,-1,0,0,NaN,NaN,NaN,"UNPLEASENT, ANTICIPATION CRITICAL, SARCASTIC",- Ajatus siitä että kaikki henkilön tienaamat...,- Ajatus siitä että kaikki henkilön tienaamat...,Meria,- Ajatus siitä että kaikki henkilön tienaamat...,- Ajatus siitä että kaikki henkilön tienaamat...,866
1,2,"- Kaivovertauksessa, viime hallitus on kaivan...",2,-2,0,0,NaN,NaN,NaN,"NEUTRAL NONE, UNPLEASENT, ANTICIPATION CRITICAL","- Kaivovertauksessa, viime hallitus on kaivan...","- Kaivovertauksessa, viime hallitus on kaivan...",Meria,"- Kaivovertauksessa, viime hallitus on kaivan...","- Kaivovertauksessa, viime hallitus on kaivan...",984
2,3,--´973´¤-.ttu,2,-5,1,2,NONE,OTHER,"SWEARING, GRANULATED",CONTEMPT DISRESPECT,--´973´¤-.ttu,973 vittu,Meria,--´973´¤-.ttu,973 vittu,9
3,4,-=>Widdu joo<=-,2,-4,1,2,NONE,TROLLING,"JOKE SARCASM, SWEARING, GRANULATED",SARCASTIC,-=> Widdu joo <=-,-=> vittu joo <=-,Meria,-=>Widdu joo<=-,-=> vittu joo <=-,17
4,5,"- Ei , mutta olen joutunut elämään katsomalla ...",2,-5,0,0,NaN,NaN,NaN,"SADNESS, FEAR","- Ei , mutta olen joutunut elämään katsomalla ...","Ei , mutta olen joutunut elämään katsomalla ku...",Meria,"- Ei , mutta olen joutunut elämään katsomalla ...","Ei , mutta olen joutunut elämään katsomalla ku...",144
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6624,6625,😂😂😂😂😂😂😂😂😂😂😂😂😂😂,1,5,0,0,NaN,NaN,NaN,JOY,😂😂😂😂😂😂😂😂😂😂😂😂😂😂,😂😂😂😂😂,Meria,😂😂😂😂😂😂😂😂😂😂😂😂😂😂,😂😂😂😂😂,5
6625,6626,😠👉👩‍💼,2,-3,0,0,NaN,NaN,NaN,ANGER HATE,😠👉👩‍💼,😠👉👩‍💼,Meria,😠👉👩‍💼,😠👉👩‍💼,5
6626,6627,🤣🤣🤣🤣🤣🤣🤣🤣🤣🤣,1,5,0,0,NaN,NaN,NaN,JOY,🤣🤣🤣🤣🤣🤣🤣🤣🤣🤣,🤣🤣🤣🤣🤣🤣🤣🤣🤣🤣,Meria,🤣🤣🤣🤣🤣🤣🤣🤣🤣🤣,🤣🤣🤣🤣🤣🤣🤣🤣🤣🤣,10
6627,6628,🤮😡,2,-4,0,0,NaN,NaN,NaN,"DISGUST, ANGER HATE",🤮😡,🤮😡,Meria,🤮😡,🤮😡,2


In [58]:
# Tokenization of raw samples with casual tokenization
df['sampleTokens'] = df['stringSample'].apply(lambda row: casual_tokenize(row))

# Tokenization of corrected samples
df['correctTokens'] = df['stringCorrect'].apply(lambda row: casual_tokenize(row))
df.head(10)

,id,sample,sentiment,polarity,HSbinary,HSstrength,HStarget,HStopic,HSform,emotion,urban_finnish,correct_finnish,user_nick,stringSample,stringCorrect,stringLength,sampleTokens,correctTokens
0,1,- Ajatus siitä että kaikki henkilön tienaamat...,2,-1,0,0,NaN,NaN,NaN,"UNPLEASENT, ANTICIPATION CRITICAL, SARCASTIC",- Ajatus siitä että kaikki henkilön tienaamat...,- Ajatus siitä että kaikki henkilön tienaamat...,Meria,- Ajatus siitä että kaikki henkilön tienaamat...,- Ajatus siitä että kaikki henkilön tienaamat...,866,"[-, Ajatus, siitä, että, kaikki, henkilön, tie...","[-, Ajatus, siitä, että, kaikki, henkilön, tie..."
1,2,"- Kaivovertauksessa, viime hallitus on kaivan...",2,-2,0,0,NaN,NaN,NaN,"NEUTRAL NONE, UNPLEASENT, ANTICIPATION CRITICAL","- Kaivovertauksessa, viime hallitus on kaivan...","- Kaivovertauksessa, viime hallitus on kaivan...",Meria,"- Kaivovertauksessa, viime hallitus on kaivan...","- Kaivovertauksessa, viime hallitus on kaivan...",984,"[-, Kaivovertauksessa, ,, viime, hallitus, on,...","[-, Kaivovertauksessa, ,, viime, hallitus, on,..."
2,3,--´973´¤-.ttu,2,-5,1,2,NONE,OTHER,"SWEARING, GRANULATED",CONTEMPT DISRESPECT,--´973´¤-.ttu,973 vittu,Meria,--´973´¤-.ttu,973 vittu,9,"[-, -, ´, 973, ´, ¤, -, ., ttu]","[973, vittu]"
3,4,-=>Widdu joo<=-,2,-4,1,2,NONE,TROLLING,"JOKE SARCASM, SWEARING, GRANULATED",SARCASTIC,-=> Widdu joo <=-,-=> vittu joo <=-,Meria,-=>Widdu joo<=-,-=> vittu joo <=-,17,"[-, =, >, Widdu, joo, <, =, -]","[-, =, >, vittu, joo, <, =, -]"
4,5,"- Ei , mutta olen joutunut elämään katsomalla ...",2,-5,0,0,NaN,NaN,NaN,"SADNESS, FEAR","- Ei , mutta olen joutunut elämään katsomalla ...","Ei , mutta olen joutunut elämään katsomalla ku...",Meria,"- Ei , mutta olen joutunut elämään katsomalla ...","Ei , mutta olen joutunut elämään katsomalla ku...",144,"[-, Ei, ,, mutta, olen, joutunut, elämään, kat...","[Ei, ,, mutta, olen, joutunut, elämään, katsom..."
5,6,- ei viddu -,2,-5,1,2,NONE,"OPINION, TROLLING","SWEARING, GRANULATED",SURPRISE,ei viddu,ei vittu,Meria,- ei viddu -,ei vittu,8,"[-, ei, viddu, -]","[ei, vittu]"
6,7,- Huvittunut -,1,1,0,0,NaN,NaN,NaN,JOY,Huvittunut,huvittunut,Meria,- Huvittunut -,huvittunut,10,"[-, Huvittunut, -]",[huvittunut]
7,8,"- Hyvä että JOKU näkee uuden , tyhmän veroperu...",2,-4,1,1,NONE,OPINION,SWEARING,"MIXED EMOTIONS, JOY, ANTICIPATION CRITICAL","- Hyvä että JOKU näkee uuden , tyhmän veroperu...","Hyvä että JOKU näkee uuden, tyhmän veroperuste...",Meria,"- Hyvä että JOKU näkee uuden , tyhmän veroperu...","Hyvä että JOKU näkee uuden, tyhmän veroperuste...",124,"[-, Hyvä, että, JOKU, näkee, uuden, ,, tyhmän,...","[Hyvä, että, JOKU, näkee, uuden, ,, tyhmän, ve..."
8,9,- Kukaan luterilainen ei korota itseään hänen ...,2,-2,0,0,NaN,NaN,NaN,SADNESS,- Kukaan luterilainen ei korota itseään hänen ...,Kukaan luterilainen ei korota itseään hänen su...,Meria,- Kukaan luterilainen ei korota itseään hänen ...,Kukaan luterilainen ei korota itseään hänen su...,105,"[-, Kukaan, luterilainen, ei, korota, itseään,...","[Kukaan, luterilainen, ei, korota, itseään, hä..."
9,10,- kuten on sovittu -,0,0,0,0,NaN,NaN,NaN,NEUTRAL NONE,- kuten on sovittu -,kuten on sovittu,Meria,- kuten on sovittu -,kuten on sovittu,16,"[-, kuten, on, sovittu, -]","[kuten, on, sovittu]"


In [59]:
# Tokenization of raw samples with casual tokenization
df['sampleTokens'] = df['sampleTokens'].apply(lambda row: ' '.join([token.lower() for token in row]))

# Tokenization of corrected samples
df['correctTokens'] = df['correctTokens'].apply(lambda row: ' '.join([token.lower() for token in row]))
df.head(10)

,id,sample,sentiment,polarity,HSbinary,HSstrength,HStarget,HStopic,HSform,emotion,urban_finnish,correct_finnish,user_nick,stringSample,stringCorrect,stringLength,sampleTokens,correctTokens
0,1,- Ajatus siitä että kaikki henkilön tienaamat...,2,-1,0,0,NaN,NaN,NaN,"UNPLEASENT, ANTICIPATION CRITICAL, SARCASTIC",- Ajatus siitä että kaikki henkilön tienaamat...,- Ajatus siitä että kaikki henkilön tienaamat...,Meria,- Ajatus siitä että kaikki henkilön tienaamat...,- Ajatus siitä että kaikki henkilön tienaamat...,866,- ajatus siitä että kaikki henkilön tienaamat ...,- ajatus siitä että kaikki henkilön tienaamat ...
1,2,"- Kaivovertauksessa, viime hallitus on kaivan...",2,-2,0,0,NaN,NaN,NaN,"NEUTRAL NONE, UNPLEASENT, ANTICIPATION CRITICAL","- Kaivovertauksessa, viime hallitus on kaivan...","- Kaivovertauksessa, viime hallitus on kaivan...",Meria,"- Kaivovertauksessa, viime hallitus on kaivan...","- Kaivovertauksessa, viime hallitus on kaivan...",984,"- kaivovertauksessa , viime hallitus on kaivan...","- kaivovertauksessa , viime hallitus on kaivan..."
2,3,--´973´¤-.ttu,2,-5,1,2,NONE,OTHER,"SWEARING, GRANULATED",CONTEMPT DISRESPECT,--´973´¤-.ttu,973 vittu,Meria,--´973´¤-.ttu,973 vittu,9,- - ´ 973 ´ ¤ - . ttu,973 vittu
3,4,-=>Widdu joo<=-,2,-4,1,2,NONE,TROLLING,"JOKE SARCASM, SWEARING, GRANULATED",SARCASTIC,-=> Widdu joo <=-,-=> vittu joo <=-,Meria,-=>Widdu joo<=-,-=> vittu joo <=-,17,- = > widdu joo < = -,- = > vittu joo < = -
4,5,"- Ei , mutta olen joutunut elämään katsomalla ...",2,-5,0,0,NaN,NaN,NaN,"SADNESS, FEAR","- Ei , mutta olen joutunut elämään katsomalla ...","Ei , mutta olen joutunut elämään katsomalla ku...",Meria,"- Ei , mutta olen joutunut elämään katsomalla ...","Ei , mutta olen joutunut elämään katsomalla ku...",144,"- ei , mutta olen joutunut elämään katsomalla ...","ei , mutta olen joutunut elämään katsomalla ku..."
5,6,- ei viddu -,2,-5,1,2,NONE,"OPINION, TROLLING","SWEARING, GRANULATED",SURPRISE,ei viddu,ei vittu,Meria,- ei viddu -,ei vittu,8,- ei viddu -,ei vittu
6,7,- Huvittunut -,1,1,0,0,NaN,NaN,NaN,JOY,Huvittunut,huvittunut,Meria,- Huvittunut -,huvittunut,10,- huvittunut -,huvittunut
7,8,"- Hyvä että JOKU näkee uuden , tyhmän veroperu...",2,-4,1,1,NONE,OPINION,SWEARING,"MIXED EMOTIONS, JOY, ANTICIPATION CRITICAL","- Hyvä että JOKU näkee uuden , tyhmän veroperu...","Hyvä että JOKU näkee uuden, tyhmän veroperuste...",Meria,"- Hyvä että JOKU näkee uuden , tyhmän veroperu...","Hyvä että JOKU näkee uuden, tyhmän veroperuste...",124,"- hyvä että joku näkee uuden , tyhmän veroperu...","hyvä että joku näkee uuden , tyhmän veroperust..."
8,9,- Kukaan luterilainen ei korota itseään hänen ...,2,-2,0,0,NaN,NaN,NaN,SADNESS,- Kukaan luterilainen ei korota itseään hänen ...,Kukaan luterilainen ei korota itseään hänen su...,Meria,- Kukaan luterilainen ei korota itseään hänen ...,Kukaan luterilainen ei korota itseään hänen su...,105,- kukaan luterilainen ei korota itseään hänen ...,kukaan luterilainen ei korota itseään hänen su...
9,10,- kuten on sovittu -,0,0,0,0,NaN,NaN,NaN,NEUTRAL NONE,- kuten on sovittu -,kuten on sovittu,Meria,- kuten on sovittu -,kuten on sovittu,16,- kuten on sovittu -,kuten on sovittu


In [60]:
# Nxgrams of raw samples
df['sampleN2grams'] = df['sampleTokens'].str.split().apply(lambda row: list(map(' '.join, ngrams(row, n=2))))
df['sampleN3grams'] = df['sampleTokens'].str.split().apply(lambda row: list(map(' '.join, ngrams(row, n=3))))
df['sampleN4grams'] = df['sampleTokens'].str.split().apply(lambda row: list(map(' '.join, ngrams(row, n=4))))
df['sampleN5grams'] = df['sampleTokens'].str.split().apply(lambda row: list(map(' '.join, ngrams(row, n=5))))

# Nxgrams of corrected samples
df['correctN2grams'] = df['correctTokens'].str.split().apply(lambda row: list(map(' '.join, ngrams(row, n=2))))
df['correctN3grams'] = df['correctTokens'].str.split().apply(lambda row: list(map(' '.join, ngrams(row, n=3))))
df['correctN4grams'] = df['correctTokens'].str.split().apply(lambda row: list(map(' '.join, ngrams(row, n=4))))
df['correctN5grams'] = df['correctTokens'].str.split().apply(lambda row: list(map(' '.join, ngrams(row, n=5))))
df.head(10)

,id,sample,sentiment,polarity,HSbinary,HSstrength,HStarget,HStopic,HSform,emotion,...,sampleTokens,correctTokens,sampleN2grams,sampleN3grams,sampleN4grams,sampleN5grams,correctN2grams,correctN3grams,correctN4grams,correctN5grams
0,1,- Ajatus siitä että kaikki henkilön tienaamat...,2,-1,0,0,NaN,NaN,NaN,"UNPLEASENT, ANTICIPATION CRITICAL, SARCASTIC",...,- ajatus siitä että kaikki henkilön tienaamat ...,- ajatus siitä että kaikki henkilön tienaamat ...,"[- ajatus, ajatus siitä, siitä että, että kaik...","[- ajatus siitä, ajatus siitä että, siitä että...","[- ajatus siitä että, ajatus siitä että kaikki...","[- ajatus siitä että kaikki, ajatus siitä että...","[- ajatus, ajatus siitä, siitä että, että kaik...","[- ajatus siitä, ajatus siitä että, siitä että...","[- ajatus siitä että, ajatus siitä että kaikki...","[- ajatus siitä että kaikki, ajatus siitä että..."
1,2,"- Kaivovertauksessa, viime hallitus on kaivan...",2,-2,0,0,NaN,NaN,NaN,"NEUTRAL NONE, UNPLEASENT, ANTICIPATION CRITICAL",...,"- kaivovertauksessa , viime hallitus on kaivan...","- kaivovertauksessa , viime hallitus on kaivan...","[- kaivovertauksessa, kaivovertauksessa ,, , v...","[- kaivovertauksessa ,, kaivovertauksessa , vi...","[- kaivovertauksessa , viime, kaivovertauksess...","[- kaivovertauksessa , viime hallitus, kaivove...","[- kaivovertauksessa, kaivovertauksessa ,, , v...","[- kaivovertauksessa ,, kaivovertauksessa , vi...","[- kaivovertauksessa , viime, kaivovertauksess...","[- kaivovertauksessa , viime hallitus, kaivove..."
2,3,--´973´¤-.ttu,2,-5,1,2,NONE,OTHER,"SWEARING, GRANULATED",CONTEMPT DISRESPECT,...,- - ´ 973 ´ ¤ - . ttu,973 vittu,"[- -, - ´, ´ 973, 973 ´, ´ ¤, ¤ -, - ., . ttu]","[- - ´, - ´ 973, ´ 973 ´, 973 ´ ¤, ´ ¤ -, ¤ - ...","[- - ´ 973, - ´ 973 ´, ´ 973 ´ ¤, 973 ´ ¤ -, ´...","[- - ´ 973 ´, - ´ 973 ´ ¤, ´ 973 ´ ¤ -, 973 ´ ...",[973 vittu],[],[],[]
3,4,-=>Widdu joo<=-,2,-4,1,2,NONE,TROLLING,"JOKE SARCASM, SWEARING, GRANULATED",SARCASTIC,...,- = > widdu joo < = -,- = > vittu joo < = -,"[- =, = >, > widdu, widdu joo, joo <, < =, = -]","[- = >, = > widdu, > widdu joo, widdu joo <, j...","[- = > widdu, = > widdu joo, > widdu joo <, wi...","[- = > widdu joo, = > widdu joo <, > widdu joo...","[- =, = >, > vittu, vittu joo, joo <, < =, = -]","[- = >, = > vittu, > vittu joo, vittu joo <, j...","[- = > vittu, = > vittu joo, > vittu joo <, vi...","[- = > vittu joo, = > vittu joo <, > vittu joo..."
4,5,"- Ei , mutta olen joutunut elämään katsomalla ...",2,-5,0,0,NaN,NaN,NaN,"SADNESS, FEAR",...,"- ei , mutta olen joutunut elämään katsomalla ...","ei , mutta olen joutunut elämään katsomalla ku...","[- ei, ei ,, , mutta, mutta olen, olen joutunu...","[- ei ,, ei , mutta, , mutta olen, mutta olen ...","[- ei , mutta, ei , mutta olen, , mutta olen j...","[- ei , mutta olen, ei , mutta olen joutunut, ...","[ei ,, , mutta, mutta olen, olen joutunut, jou...","[ei , mutta, , mutta olen, mutta olen joutunut...","[ei , mutta olen, , mutta olen joutunut, mutta...","[ei , mutta olen joutunut, , mutta olen joutun..."
5,6,- ei viddu -,2,-5,1,2,NONE,"OPINION, TROLLING","SWEARING, GRANULATED",SURPRISE,...,- ei viddu -,ei vittu,"[- ei, ei viddu, viddu -]","[- ei viddu, ei viddu -]",[- ei viddu -],[],[ei vittu],[],[],[]
6,7,- Huvittunut -,1,1,0,0,NaN,NaN,NaN,JOY,...,- huvittunut -,huvittunut,"[- huvittunut, huvittunut -]",[- huvittunut -],[],[],[],[],[],[]
7,8,"- Hyvä että JOKU näkee uuden , tyhmän veroperu...",2,-4,1,1,NONE,OPINION,SWEARING,"MIXED EMOTIONS, JOY, ANTICIPATION CRITICAL",...,"- hyvä että joku näkee uuden , tyhmän veroperu...","hyvä että joku näkee uuden , tyhmän veroperust...","[- hyvä, hyvä että, että joku, joku näkee, näk...","[- hyvä että, hyvä että joku, että joku näkee,...","[- hyvä että joku, hyvä että joku näkee, että ...","[- hyvä että joku näkee, hyvä että joku näkee ...","[hyvä että, että joku, joku näkee, näkee uuden...","[hyvä että joku, että joku näkee, joku näkee u...","[hyvä että joku näkee, että joku näkee uuden, ...","[hyvä että joku näkee uu

### N2grams

In [61]:
# N2grams of tokenized raw samples

all_n2grams = []
for sample in df['sampleN2grams']:
    for item in sample:
        all_n2grams.append(item)
all_n2grams

#count the lemma twograms
N2gram = Counter(all_n2grams)
N2gram

Counter({'- ajatus': 1,
         'ajatus siitä': 1,
         'siitä että': 8,
         'että kaikki': 8,
         'kaikki henkilön': 1,
         'henkilön tienaamat': 1,
         'tienaamat tulot': 1,
         'tulot ovat': 1,
         'ovat hänen': 1,
         'hänen oman': 1,
         'oman työnsä': 1,
         'työnsä tulosta': 1,
         'tulosta on': 1,
         'on lähinnä': 1,
         'lähinnä naurettava': 1,
         'naurettava .': 1,
         '. se': 5,
         'se voisi': 1,
         'voisi ehkä': 2,
         'ehkä päteä': 1,
         'päteä ,': 1,
         ', jos': 57,
         'jos henkilö': 1,
         'henkilö eläisi': 1,
         'eläisi yksin': 1,
         'yksin metsässä': 1,
         'metsässä susien': 1,
         'susien kasvattamana': 1,
         'kasvattamana ,': 1,
         ', mutta': 156,
         'mutta ei': 18,
         'ei modernissa': 1,
         'modernissa hyvinvointiyhteiskunnassa': 1,
         'hyvinvointiyhteiskunnassa .': 1,
         '. yhteiskunta'

In [62]:
# create data frame for lemmas
lemma_freq_N2 = pd.DataFrame(N2gram.items(),columns=['lemma','frequency']).sort_values(by='frequency',ascending=False)

# load dataframe to csv at at output folder
dfresults = lemma_freq_N2

def load_N2grams_toCSV(df):
    # set relative path to folder
    cwd = os.getcwd()
    folder = '\\results\\'
    # set file
    csv_file = 'fin_lemma_N2grams_forRawSamples_csvtable_'
    #timestr = time.strftime("%Y%m%d-%H%M%S")
    csv_postfix = '.csv'
    #csv_source = cwd + folder + csv_file + timestr + csv_postfix
    csv_source = cwd + folder + csv_file + csv_postfix
    # get dataframe to CSV at defined source file ‘utf-8’
    df.to_csv(csv_source, index=False, encoding="utf-8-sig")

# Load dataframe to json at output folder
def load_N2grams_toJSON(df):
    # set relative path to folder
    cwd = os.getcwd()
    folder = '\\results\\'
    # set file
    json_file = 'fin_lemma_N2grams_forRawSamples_JSON_'
    #timestr = time.strftime("%Y%m%d-%H%M%S")
    json_postfix = '.json'
    #json_source = cwd + folder + json_file + timestr + json_postfix
    json_source = cwd + folder + json_file + json_postfix
    # get dataframe to json 
    result = df.to_json(orient='records')
    parsed = json.loads(result)
    parsed = {"fin_lemma_N2grams_forRawSamples": parsed}
    # Writing parsed data to json file
    json_object = json.dumps(parsed)
    with open(json_source, "w") as outfile:
        outfile.write(json_object)
        
# Load dataframe to excel and csv at defined source file
load_N2grams_toCSV(dfresults)
load_N2grams_toJSON(dfresults)

lemma_freq_N2.head(50)

,lemma,frequency
508,! !,437
2961,vittu mikä,271
420,", että",246
2891,:d :d,205
1688,vittu mitä,172
29,", mutta",156
223,ei ole,115
2264,ei vittu,103
355,", kun",93
17539,ime paskaa,85


### N3grams 

In [63]:
# N3grams of tokenized raw samples

all_n3grams = []
for sample in df['sampleN3grams']:
    for item in sample:
        all_n3grams.append(item)
all_n3grams

#count the lemma twograms
N3gram = Counter(all_n3grams)
N3gram

Counter({'- ajatus siitä': 1,
         'ajatus siitä että': 1,
         'siitä että kaikki': 1,
         'että kaikki henkilön': 1,
         'kaikki henkilön tienaamat': 1,
         'henkilön tienaamat tulot': 1,
         'tienaamat tulot ovat': 1,
         'tulot ovat hänen': 1,
         'ovat hänen oman': 1,
         'hänen oman työnsä': 1,
         'oman työnsä tulosta': 1,
         'työnsä tulosta on': 1,
         'tulosta on lähinnä': 1,
         'on lähinnä naurettava': 1,
         'lähinnä naurettava .': 1,
         'naurettava . se': 1,
         '. se voisi': 1,
         'se voisi ehkä': 1,
         'voisi ehkä päteä': 1,
         'ehkä päteä ,': 1,
         'päteä , jos': 1,
         ', jos henkilö': 1,
         'jos henkilö eläisi': 1,
         'henkilö eläisi yksin': 1,
         'eläisi yksin metsässä': 1,
         'yksin metsässä susien': 1,
         'metsässä susien kasvattamana': 1,
         'susien kasvattamana ,': 1,
         'kasvattamana , mutta': 1,
         ', mutta

In [64]:
# create data frame for lemmas
lemma_freq_N3 = pd.DataFrame(N3gram.items(),columns=['lemma','frequency']).sort_values(by='frequency',ascending=False)

# load dataframe to csv at at output folder
dfresults = lemma_freq_N3

def load_N3grams_toCSV(df):
    # set relative path to folder
    cwd = os.getcwd()
    folder = '\\results\\'
    # set file
    csv_file = 'fin_lemma_N3grams_forRawSamples_csvtable_'
    #timestr = time.strftime("%Y%m%d-%H%M%S")
    csv_postfix = '.csv'
    #csv_source = cwd + folder + csv_file + timestr + csv_postfix
    csv_source = cwd + folder + csv_file + csv_postfix
    # get dataframe to CSV at defined source file ‘utf-8’
    df.to_csv(csv_source, index=False, encoding="utf-8-sig")

# Load dataframe to json at output folder
def load_N3grams_toJSON(df):
    # set relative path to folder
    cwd = os.getcwd()
    folder = '\\results\\'
    # set file
    json_file = 'fin_lemma_N3grams_forRawSamples_JSON_'
    #timestr = time.strftime("%Y%m%d-%H%M%S")
    json_postfix = '.json'
    #json_source = cwd + folder + json_file + timestr + json_postfix
    json_source = cwd + folder + json_file + json_postfix
    # get dataframe to json 
    result = df.to_json(orient='records')
    parsed = json.loads(result)
    parsed = {"fin_lemma_N3grams_forRawSamples": parsed}
    # Writing parsed data to json file
    json_object = json.dumps(parsed)
    with open(json_source, "w") as outfile:
        outfile.write(json_object)
        
# Load dataframe to excel and csv at defined source file
load_N3grams_toCSV(dfresults)
load_N3grams_toJSON(dfresults)

lemma_freq_N3.head(50)

,lemma,frequency
510,! ! !,260
8075,:d :d :d,197
18007,ime paskaa ime,84
18008,paskaa ime paskaa,84
7391,ei vittu mikä,20
517,jossa on noobit,19
511,! ! mopar,19
512,! mopar on,19
513,mopar on paska,19
514,on paska paikka,19


### N4grams

In [65]:
# N4grams of tokenized, lemmatized and cleaned raw samples

all_n4grams = []
for sample in df['sampleN4grams']:
    for item in sample:
        all_n4grams.append(item)
all_n4grams

#count the lemma twograms
N4gram = Counter(all_n4grams)
N4gram

Counter({'- ajatus siitä että': 1,
         'ajatus siitä että kaikki': 1,
         'siitä että kaikki henkilön': 1,
         'että kaikki henkilön tienaamat': 1,
         'kaikki henkilön tienaamat tulot': 1,
         'henkilön tienaamat tulot ovat': 1,
         'tienaamat tulot ovat hänen': 1,
         'tulot ovat hänen oman': 1,
         'ovat hänen oman työnsä': 1,
         'hänen oman työnsä tulosta': 1,
         'oman työnsä tulosta on': 1,
         'työnsä tulosta on lähinnä': 1,
         'tulosta on lähinnä naurettava': 1,
         'on lähinnä naurettava .': 1,
         'lähinnä naurettava . se': 1,
         'naurettava . se voisi': 1,
         '. se voisi ehkä': 1,
         'se voisi ehkä päteä': 1,
         'voisi ehkä päteä ,': 1,
         'ehkä päteä , jos': 1,
         'päteä , jos henkilö': 1,
         ', jos henkilö eläisi': 1,
         'jos henkilö eläisi yksin': 1,
         'henkilö eläisi yksin metsässä': 1,
         'eläisi yksin metsässä susien': 1,
         'yksin 

In [66]:
# create data frame for lemmas
lemma_freq_N4 = pd.DataFrame(N4gram.items(),columns=['lemma','frequency']).sort_values(by='frequency',ascending=False)

# load dataframe to csv at at output folder
dfresults = lemma_freq_N4

def load_N4grams_toCSV(df):
    # set relative path to folder
    cwd = os.getcwd()
    folder = '\\results\\'
    # set file
    csv_file = 'fin_lemma_N4grams_forRawSamples_csvtable_'
    #timestr = time.strftime("%Y%m%d-%H%M%S")
    csv_postfix = '.csv'
    #csv_source = cwd + folder + csv_file + timestr + csv_postfix
    csv_source = cwd + folder + csv_file + csv_postfix
    # get dataframe to CSV at defined source file ‘utf-8’
    df.to_csv(csv_source, index=False, encoding="utf-8-sig")

# Load dataframe to json at output folder
def load_N4grams_toJSON(df):
    # set relative path to folder
    cwd = os.getcwd()
    folder = '\\results\\'
    # set file
    json_file = 'fin_lemma_N4grams_forRawSamples_JSON_'
    #timestr = time.strftime("%Y%m%d-%H%M%S")
    json_postfix = '.json'
    #json_source = cwd + folder + json_file + timestr + json_postfix
    json_source = cwd + folder + json_file + json_postfix
    # get dataframe to json 
    result = df.to_json(orient='records')
    parsed = json.loads(result)
    parsed = {"fin_lemma_N4grams_forRawSamples": parsed}
    # Writing parsed data to json file
    json_object = json.dumps(parsed)
    with open(json_source, "w") as outfile:
        outfile.write(json_object)
        
# Load dataframe to excel and csv at defined source file
load_N4grams_toCSV(dfresults)
load_N4grams_toJSON(dfresults)

lemma_freq_N4.head(50)

,lemma,frequency
7260,:d :d :d :d,193
506,! ! ! !,148
16607,ime paskaa ime paskaa,84
16608,paskaa ime paskaa ime,83
500,paska paikka jossa on,19
505,homot ! ! !,19
504,noobit homot ! !,19
503,on noobit homot !,19
502,jossa on noobit homot,19
501,paikka jossa on noobit,19


### N5grams

In [67]:
# N5grams of tokenized, lemmatized and cleaned raw samples

all_n5grams = []
for sample in df['sampleN5grams']:
    for item in sample:
        all_n5grams.append(item)
all_n5grams

#count the lemma twograms
N5gram = Counter(all_n5grams)
N5gram

Counter({'- ajatus siitä että kaikki': 1,
         'ajatus siitä että kaikki henkilön': 1,
         'siitä että kaikki henkilön tienaamat': 1,
         'että kaikki henkilön tienaamat tulot': 1,
         'kaikki henkilön tienaamat tulot ovat': 1,
         'henkilön tienaamat tulot ovat hänen': 1,
         'tienaamat tulot ovat hänen oman': 1,
         'tulot ovat hänen oman työnsä': 1,
         'ovat hänen oman työnsä tulosta': 1,
         'hänen oman työnsä tulosta on': 1,
         'oman työnsä tulosta on lähinnä': 1,
         'työnsä tulosta on lähinnä naurettava': 1,
         'tulosta on lähinnä naurettava .': 1,
         'on lähinnä naurettava . se': 1,
         'lähinnä naurettava . se voisi': 1,
         'naurettava . se voisi ehkä': 1,
         '. se voisi ehkä päteä': 1,
         'se voisi ehkä päteä ,': 1,
         'voisi ehkä päteä , jos': 1,
         'ehkä päteä , jos henkilö': 1,
         'päteä , jos henkilö eläisi': 1,
         ', jos henkilö eläisi yksin': 1,
         'j

In [68]:
# create data frame for lemmas
lemma_freq_N5 = pd.DataFrame(N5gram.items(),columns=['lemma','frequency']).sort_values(by='frequency',ascending=False)

# load dataframe to csv at at output folder
dfresults = lemma_freq_N5

def load_N5grams_toCSV(df):
    # set relative path to folder
    cwd = os.getcwd()
    folder = '\\results\\'
    # set file
    csv_file = 'fin_lemma_N5grams_forRawSamples_csvtable_'
    #timestr = time.strftime("%Y%m%d-%H%M%S")
    csv_postfix = '.csv'
    #csv_source = cwd + folder + csv_file + timestr + csv_postfix
    csv_source = cwd + folder + csv_file + csv_postfix
    # get dataframe to CSV at defined source file ‘utf-8’
    df.to_csv(csv_source, index=False, encoding="utf-8-sig")

# Load dataframe to json at output folder
def load_N5grams_toJSON(df):
    # set relative path to folder
    cwd = os.getcwd()
    folder = '\\results\\'
    # set file
    json_file = 'fin_lemma_N5grams_forRawSamples_JSON_'
    #timestr = time.strftime("%Y%m%d-%H%M%S")
    json_postfix = '.json'
    #json_source = cwd + folder + json_file + timestr + json_postfix
    json_source = cwd + folder + json_file + json_postfix
    # get dataframe to json 
    result = df.to_json(orient='records')
    parsed = json.loads(result)
    parsed = {"fin_lemma_N5grams_forRawSamples": parsed}
    # Writing parsed data to json file
    json_object = json.dumps(parsed)
    with open(json_source, "w") as outfile:
        outfile.write(json_object)
        
# Load dataframe to excel and csv at defined source file
load_N5grams_toCSV(dfresults)
load_N5grams_toJSON(dfresults)

lemma_freq_N5.head(50)

,lemma,frequency
25213,:d :d :d :d :d,190
6570,! ! ! ! !,118
15229,paskaa ime paskaa ime paskaa,83
15228,ime paskaa ime paskaa ime,83
480,! ! ! mopar on,19
481,! ! mopar on paska,19
482,! mopar on paska paikka,19
484,on paska paikka jossa on,19
485,paska paikka jossa on noobit,19
486,paikka jossa on noobit homot,19


# -----------------------------------------------------------------------------------
# Nxgram analysis for positive and negative class
# raw samples analysis - no cleaning, no lemmatization

In [69]:
# Nxgram analysis for positive class and negative class

df_positive = df[df['sentiment'] == 1]
#df_positive
df_negative = df[df['sentiment'] == 2]
df_negative

,id,sample,sentiment,polarity,HSbinary,HSstrength,HStarget,HStopic,HSform,emotion,...,sampleTokens,correctTokens,sampleN2grams,sampleN3grams,sampleN4grams,sampleN5grams,correctN2grams,correctN3grams,correctN4grams,correctN5grams
0,1,- Ajatus siitä että kaikki henkilön tienaamat...,2,-1,0,0,NaN,NaN,NaN,"UNPLEASENT, ANTICIPATION CRITICAL, SARCASTIC",...,- ajatus siitä että kaikki henkilön tienaamat ...,- ajatus siitä että kaikki henkilön tienaamat ...,"[- ajatus, ajatus siitä, siitä että, että kaik...","[- ajatus siitä, ajatus siitä että, siitä että...","[- ajatus siitä että, ajatus siitä että kaikki...","[- ajatus siitä että kaikki, ajatus siitä että...","[- ajatus, ajatus siitä, siitä että, että kaik...","[- ajatus siitä, ajatus siitä että, siitä että...","[- ajatus siitä että, ajatus siitä että kaikki...","[- ajatus siitä että kaikki, ajatus siitä että..."
1,2,"- Kaivovertauksessa, viime hallitus on kaivan...",2,-2,0,0,NaN,NaN,NaN,"NEUTRAL NONE, UNPLEASENT, ANTICIPATION CRITICAL",...,"- kaivovertauksessa , viime hallitus on kaivan...","- kaivovertauksessa , viime hallitus on kaivan...","[- kaivovertauksessa, kaivovertauksessa ,, , v...","[- kaivovertauksessa ,, kaivovertauksessa , vi...","[- kaivovertauksessa , viime, kaivovertauksess...","[- kaivovertauksessa , viime hallitus, kaivove...","[- kaivovertauksessa, kaivovertauksessa ,, , v...","[- kaivovertauksessa ,, kaivovertauksessa , vi...","[- kaivovertauksessa , viime, kaivovertauksess...","[- kaivovertauksessa , viime hallitus, kaivove..."
2,3,--´973´¤-.ttu,2,-5,1,2,NONE,OTHER,"SWEARING, GRANULATED",CONTEMPT DISRESPECT,...,- - ´ 973 ´ ¤ - . ttu,973 vittu,"[- -, - ´, ´ 973, 973 ´, ´ ¤, ¤ -, - ., . ttu]","[- - ´, - ´ 973, ´ 973 ´, 973 ´ ¤, ´ ¤ -, ¤ - ...","[- - ´ 973, - ´ 973 ´, ´ 973 ´ ¤, 973 ´ ¤ -, ´...","[- - ´ 973 ´, - ´ 973 ´ ¤, ´ 973 ´ ¤ -, 973 ´ ...",[973 vittu],[],[],[]
3,4,-=>Widdu joo<=-,2,-4,1,2,NONE,TROLLING,"JOKE SARCASM, SWEARING, GRANULATED",SARCASTIC,...,- = > widdu joo < = -,- = > vittu joo < = -,"[- =, = >, > widdu, widdu joo, joo <, < =, = -]","[- = >, = > widdu, > widdu joo, widdu joo <, j...","[- = > widdu, = > widdu joo, > widdu joo <, wi...","[- = > widdu joo, = > widdu joo <, > widdu joo...","[- =, = >, > vittu, vittu joo, joo <, < =, = -]","[- = >, = > vittu, > vittu joo, vittu joo <, j...","[- = > vittu, = > vittu joo, > vittu joo <, vi...","[- = > vittu joo, = > vittu joo <, > vittu joo..."
4,5,"- Ei , mutta olen joutunut elämään katsomalla ...",2,-5,0,0,NaN,NaN,NaN,"SADNESS, FEAR",...,"- ei , mutta olen joutunut elämään katsomalla ...","ei , mutta olen joutunut elämään katsomalla ku...","[- ei, ei ,, , mutta, mutta olen, olen joutunu...","[- ei ,, ei , mutta, , mutta olen, mutta olen ...","[- ei , mutta, ei , mutta olen, , mutta olen j...","[- ei , mutta olen, ei , mutta olen joutunut, ...","[ei ,, , mutta, mutta olen, olen joutunut, jou...","[ei , mutta, , mutta olen, mutta olen joutunut...","[ei , mutta olen, , mutta olen joutunut, mutta...","[ei , mutta olen joutunut, , mutta olen joutun..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6616,6617,Ööövittuentiiä,2,-5,1,2,NONE,"OPINION, TROLLING","SWEARING, GRANULATED",CONTEMPT DISRESPECT,...,ööövittuentiiä,öö vittu en tiedä,[],[],[],[],"[öö vittu, vittu en, en tiedä]","[öö vittu en, vittu en tiedä]",[öö vittu en tiedä],[]
6619,6620,👎😠,2,-3,0,0,NaN,NaN,NaN,"UNPLEASENT, ANGER HATE",...,👎 😠,👎 😠,[👎 😠],[],[],[],[👎 😠],[],[],[]
6622,6623,😂 ai kauhee. Ei oo hauska asia jos itelle tapa...,2,-4,0,0,NaN,NaN,NaN,SURPRISE,...,😂 ai kauhee . ei oo hauska asia jos itelle tap...,😂 ai kauhee . ei oo hauska asia jos itelle tap...,"[😂 ai, ai kauhee, kauhee ., . ei, ei oo, oo ha...","[😂 ai kauhee, ai kauhee ., kauhee . ei, . ei o...","[😂 ai kauhee ., ai kauhee . ei, kauhee . ei oo...","[😂 ai kauhee . ei, ai kauhee . ei oo, kauhee ....","[😂 ai, ai kauhee, kauhee ., . ei, ei oo, oo ha...","[😂 ai kauhee, ai kauhee ., kauhee . ei, . ei o...","[😂 ai kauhee ., ai kauhee . ei, kau

In [70]:
# ------------------------------------------------------#
# collect all 'positive' labeled RAW samples to one list
# Nxgram analysis of tokenized RAW samples

all_n2grams = []
for sample in df_positive['correctN2grams']:
    for word in sample:
        all_n2grams.append(word)
all_n2grams

#count the lemma twograms
N2gram = Counter(all_n2grams)
N2gram

Counter({'! huvittunut': 1,
         'huvittunut !': 2,
         '" mikset': 1,
         'mikset sinäkin': 1,
         'sinäkin ottaisi': 1,
         'ottaisi vastaan': 1,
         'vastaan timo': 1,
         'timo jutilaa': 1,
         'jutilaa ja': 1,
         'ja tapparan': 1,
         'tapparan hurmoksellista': 1,
         'hurmoksellista joukkuehenkeä': 1,
         'joukkuehenkeä !': 1,
         '! "': 1,
         '" hei': 1,
         'hei vaan': 1,
         'vaan teille': 1,
         'teille ja': 1,
         'ja onnittelut': 1,
         'onnittelut "': 1,
         '" 😂': 2,
         '😂 😂': 19,
         '😂 😭': 1,
         '" ikärajoitettu': 1,
         'ikärajoitettu video': 1,
         'video "': 1,
         '" nais-oletettuja': 1,
         'nais-oletettuja "': 1,
         '" xd': 2,
         '" näin': 1,
         'näin me': 1,
         'me vihreät': 1,
         'vihreät ajattelemme': 1,
         'ajattelemme "': 1,
         '" .': 3,
         '. hauska': 1,
         'hauska pätk

In [71]:
# create data frame for lemmas
lemma_freq_N2 = pd.DataFrame(N2gram.items(),columns=['lemma','frequency']).sort_values(by='frequency',ascending=False)

# load dataframe to csv at at output folder
dfresults = lemma_freq_N2

def load_N2grams_toCSV(df):
    # set relative path to folder
    cwd = os.getcwd()
    folder = '\\results\\'
    # set file
    csv_file = 'fin_lemma_N2grams_RAWpositiveClass_csvtable_'
    #timestr = time.strftime("%Y%m%d-%H%M%S")
    csv_postfix = '.csv'
    #csv_source = cwd + folder + csv_file + timestr + csv_postfix
    csv_source = cwd + folder + csv_file + csv_postfix
    # get dataframe to CSV at defined source file ‘utf-8’
    df.to_csv(csv_source, index=False, encoding="utf-8-sig")

# Load dataframe to json at output folder
def load_N2grams_toJSON(df):
    # set relative path to folder
    cwd = os.getcwd()
    folder = '\\results\\'
    # set file
    json_file = 'fin_lemma_N2grams_RAWpositiveClass_JSON_'
    #timestr = time.strftime("%Y%m%d-%H%M%S")
    json_postfix = '.json'
    #json_source = cwd + folder + json_file + timestr + json_postfix
    json_source = cwd + folder + json_file + json_postfix
    # get dataframe to json 
    result = df.to_json(orient='records')
    parsed = json.loads(result)
    parsed = {"fin_lemma_N2grams_RAWpositiveClass": parsed}
    # Writing parsed data to json file
    json_object = json.dumps(parsed)
    with open(json_source, "w") as outfile:
        outfile.write(json_object)
        
# Load dataframe to excel and csv at defined source file
load_N2grams_toCSV(dfresults)
load_N2grams_toJSON(dfresults)

lemma_freq_N2.head(50)

,lemma,frequency
1499,! !,27
643,", että",26
857,", ja",25
61,", mutta",20
21,😂 😂,19
262,❤ ️,16
1083,", joka",14
668,", kun",12
1024,ei vittu,12
150,", niin",12


In [72]:
# ------------------------------------------------------#
# collect all 'positive' labeled RAW samples to one list
# Nxgram analysis of tokenized RAW samples

all_n3grams = []
for sample in df_positive['correctN3grams']:
    for word in sample:
        all_n3grams.append(word)
all_n3grams

#count the lemma twograms
N3gram = Counter(all_n3grams)
N3gram

Counter({'! huvittunut !': 1,
         '" mikset sinäkin': 1,
         'mikset sinäkin ottaisi': 1,
         'sinäkin ottaisi vastaan': 1,
         'ottaisi vastaan timo': 1,
         'vastaan timo jutilaa': 1,
         'timo jutilaa ja': 1,
         'jutilaa ja tapparan': 1,
         'ja tapparan hurmoksellista': 1,
         'tapparan hurmoksellista joukkuehenkeä': 1,
         'hurmoksellista joukkuehenkeä !': 1,
         'joukkuehenkeä ! "': 1,
         '" hei vaan': 1,
         'hei vaan teille': 1,
         'vaan teille ja': 1,
         'teille ja onnittelut': 1,
         'ja onnittelut "': 1,
         'onnittelut " 😂': 1,
         '" 😂 😂': 2,
         '😂 😂 😭': 1,
         '" ikärajoitettu video': 1,
         'ikärajoitettu video "': 1,
         'video " 😂': 1,
         '" nais-oletettuja "': 1,
         'nais-oletettuja " xd': 1,
         '" näin me': 1,
         'näin me vihreät': 1,
         'me vihreät ajattelemme': 1,
         'vihreät ajattelemme "': 1,
         'ajattelemme 

In [73]:
# create data frame for lemmas
lemma_freq_N3 = pd.DataFrame(N3gram.items(),columns=['lemma','frequency']).sort_values(by='frequency',ascending=False)

# load dataframe to csv at at output folder
dfresults = lemma_freq_N3

def load_N3grams_toCSV(df):
    # set relative path to folder
    cwd = os.getcwd()
    folder = '\\results\\'
    # set file
    csv_file = 'fin_lemma_N3grams_RAWpositiveClass_csvtable_'
    #timestr = time.strftime("%Y%m%d-%H%M%S")
    csv_postfix = '.csv'
    #csv_source = cwd + folder + csv_file + timestr + csv_postfix
    csv_source = cwd + folder + csv_file + csv_postfix
    # get dataframe to CSV at defined source file ‘utf-8’
    df.to_csv(csv_source, index=False, encoding="utf-8-sig")

# Load dataframe to json at output folder
def load_N3grams_toJSON(df):
    # set relative path to folder
    cwd = os.getcwd()
    folder = '\\results\\'
    # set file
    json_file = 'fin_lemma_N3grams_RAWpositiveClass_JSON_'
    #timestr = time.strftime("%Y%m%d-%H%M%S")
    json_postfix = '.json'
    #json_source = cwd + folder + json_file + timestr + json_postfix
    json_source = cwd + folder + json_file + json_postfix
    # get dataframe to json 
    result = df.to_json(orient='records')
    parsed = json.loads(result)
    parsed = {"fin_lemma_RAWpositiveClass": parsed}
    # Writing parsed data to json file
    json_object = json.dumps(parsed)
    with open(json_source, "w") as outfile:
        outfile.write(json_object)
        
# Load dataframe to excel and csv at defined source file
load_N3grams_toCSV(dfresults)
load_N3grams_toJSON(dfresults)

lemma_freq_N3.head(50)

,lemma,frequency
924,ei vittu mage,11
3964,😂 😂 😂,10
238,️ ❤ ️,8
2537,! ! !,8
237,❤ ️ ❤,7
1087,kuin vaan voi,6
3715,", , ,",6
888,elämä on ihanaa,5
1088,vaan voi :),5
348,on aivan ihana,5


In [74]:
# ------------------------------------------------------#
# collect all 'positive' labeled RAW samples to one list
# Nxgram analysis of tokenized RAW samples

all_n4grams = []
for sample in df_positive['correctN4grams']:
    for word in sample:
        all_n4grams.append(word)
all_n4grams

#count the lemma twograms
N4gram = Counter(all_n4grams)
N4gram

Counter({'" mikset sinäkin ottaisi': 1,
         'mikset sinäkin ottaisi vastaan': 1,
         'sinäkin ottaisi vastaan timo': 1,
         'ottaisi vastaan timo jutilaa': 1,
         'vastaan timo jutilaa ja': 1,
         'timo jutilaa ja tapparan': 1,
         'jutilaa ja tapparan hurmoksellista': 1,
         'ja tapparan hurmoksellista joukkuehenkeä': 1,
         'tapparan hurmoksellista joukkuehenkeä !': 1,
         'hurmoksellista joukkuehenkeä ! "': 1,
         '" hei vaan teille': 1,
         'hei vaan teille ja': 1,
         'vaan teille ja onnittelut': 1,
         'teille ja onnittelut "': 1,
         'ja onnittelut " 😂': 1,
         'onnittelut " 😂 😂': 1,
         '" 😂 😂 😭': 1,
         '" ikärajoitettu video "': 1,
         'ikärajoitettu video " 😂': 1,
         'video " 😂 😂': 1,
         '" nais-oletettuja " xd': 1,
         '" näin me vihreät': 1,
         'näin me vihreät ajattelemme': 1,
         'me vihreät ajattelemme "': 1,
         'vihreät ajattelemme " .': 1,
      

In [75]:
# create data frame for lemmas
lemma_freq_N4 = pd.DataFrame(N4gram.items(),columns=['lemma','frequency']).sort_values(by='frequency',ascending=False)

# load dataframe to csv at output folder
dfresults = lemma_freq_N4

def load_N4grams_toCSV(df):
    # set relative path to folder
    cwd = os.getcwd()
    folder = '\\results\\'
    # set file
    csv_file = 'fin_lemma_N4grams_RAWpositiveClass_csvtable_'
    #timestr = time.strftime("%Y%m%d-%H%M%S")
    csv_postfix = '.csv'
    #csv_source = cwd + folder + csv_file + timestr + csv_postfix
    csv_source = cwd + folder + csv_file + csv_postfix
    # get dataframe to CSV at defined source file ‘utf-8’
    df.to_csv(csv_source, index=False, encoding="utf-8-sig")

# Load dataframe to json at output folder
def load_N4grams_toJSON(df):
    # set relative path to folder
    cwd = os.getcwd()
    folder = '\\results\\'
    # set file
    json_file = 'fin_lemma_N4grams_RAWpositiveClass_JSON_'
    #timestr = time.strftime("%Y%m%d-%H%M%S")
    json_postfix = '.json'
    #json_source = cwd + folder + json_file + timestr + json_postfix
    json_source = cwd + folder + json_file + json_postfix
    # get dataframe to json 
    result = df.to_json(orient='records')
    parsed = json.loads(result)
    parsed = {"fin_lemma_RAWpositiveClass": parsed}
    # Writing parsed data to json file
    json_object = json.dumps(parsed)
    with open(json_source, "w") as outfile:
        outfile.write(json_object)
        
# Load dataframe to excel and csv at defined source file
load_N4grams_toCSV(dfresults)
load_N4grams_toJSON(dfresults)

print('Length of result table: ', len(dfresults))
lemma_freq_N4.head(50)

Length of result table:  6151


,lemma,frequency
210,❤ ️ ❤ ️,7
978,kuin vaan voi :),5
3659,😂 😂 😂 😂,5
211,️ ❤ ️ ❤,5
829,mage : - -,3
828,vittu mage : -,3
827,ei vittu mage :,3
4117,:d :d :d :d,3
1372,", aurinkoisinta , ihaninta",2
1370,"maailman onnellisinta , aurinkoisinta",2


In [76]:
# ------------------------------------------------------#
# collect all 'positive' labeled RAW samples to one list
# Nxgram analysis of tokenized RAW samples

all_n5grams = []
for sample in df_positive['sampleN5grams']:
    for item in sample:
        all_n5grams.append(item)
all_n5grams

#count the lemma twograms
N5gram = Counter(all_n5grams)
N5gram

Counter({'" mikset sinäkin ottaisi vastaan': 1,
         'mikset sinäkin ottaisi vastaan timo': 1,
         'sinäkin ottaisi vastaan timo jutilaa': 1,
         'ottaisi vastaan timo jutilaa ja': 1,
         'vastaan timo jutilaa ja tapparan': 1,
         'timo jutilaa ja tapparan hurmoksellista': 1,
         'jutilaa ja tapparan hurmoksellista joukkuehenkeä': 1,
         'ja tapparan hurmoksellista joukkuehenkeä !': 1,
         'tapparan hurmoksellista joukkuehenkeä ! "': 1,
         '" hei vaan teille ja': 1,
         'hei vaan teille ja onnittelut': 1,
         'vaan teille ja onnittelut "': 1,
         'teille ja onnittelut " 😂': 1,
         'ja onnittelut " 😂 😂': 1,
         'onnittelut " 😂 😂 😭': 1,
         '" ikärajoitettu video " 😂': 1,
         'ikärajoitettu video " 😂 😂': 1,
         '" näin me vihreät ajattelemme': 1,
         'näin me vihreät ajattelemme "': 1,
         'me vihreät ajattelemme " .': 1,
         'vihreät ajattelemme " . hauska': 1,
         'ajattelemme " . h

In [77]:
# create data frame for lemmas
lemma_freq_N5 = pd.DataFrame(N5gram.items(),columns=['lemma','frequency']).sort_values(by='frequency',ascending=False)

# load dataframe to csv at at output folder
dfresults = lemma_freq_N5

def load_N5grams_toCSV(df):
    # set relative path to folder
    cwd = os.getcwd()
    folder = '\\results\\'
    # set file
    csv_file = 'fin_lemma_N5grams_RAWpositiveClass_csvtable_'
    #timestr = time.strftime("%Y%m%d-%H%M%S")
    csv_postfix = '.csv'
    #csv_source = cwd + folder + csv_file + timestr + csv_postfix
    csv_source = cwd + folder + csv_file + csv_postfix
    # get dataframe to CSV at defined source file ‘utf-8’
    df.to_csv(csv_source, index=False, encoding="utf-8-sig")

# Load dataframe to json at output folder
def load_N5grams_toJSON(df):
    # set relative path to folder
    cwd = os.getcwd()
    folder = '\\results\\'
    # set file
    json_file = 'fin_lemma_N5grams_RAWpositiveClass_JSON_'
    #timestr = time.strftime("%Y%m%d-%H%M%S")
    json_postfix = '.json'
    #json_source = cwd + folder + json_file + timestr + json_postfix
    json_source = cwd + folder + json_file + json_postfix
    # get dataframe to json 
    result = df.to_json(orient='records')
    parsed = json.loads(result)
    parsed = {"fin_lemma_N5grams_RAWpositiveClass": parsed}
    # Writing parsed data to json file
    json_object = json.dumps(parsed)
    with open(json_source, "w") as outfile:
        outfile.write(json_object)
        
# Load dataframe to excel and csv at defined source file
load_N5grams_toCSV(dfresults)
load_N5grams_toJSON(dfresults)

lemma_freq_N5.head(50)

,lemma,frequency
2127,! ! ! ! !,18
190,️ ❤ ️ ❤ ️,5
189,❤ ️ ❤ ️ ❤,4
3288,😂 😂 😂 😂 😂,4
730,viddu mage : - -,3
729,ei viddu mage : -,3
1224,ihaninta ja turvallisinta yhteistä maailmaa,2
243,30 - v : huvittunut,2
912,on ihanaa kun ei tarvitse,2
1144,ihanaa mitä yhdessä keksimme :),2
